# [CONFIG] Configuração básica

## Básicos (numpy, pandas, math, display, locale, time, random, json)

In [510]:
!python -m pip install pandas
!python -m pip install openpyxl
!python -m pip install xlrd
!python -m pip install jupyter
!python -m pip install numpy
# !python -m pip install random
!python -m pip install IPython
!python -m pip install json


from IPython.display import display
import json
import pandas as pd
import math
import numpy as np
import locale
locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")  # Use "" for auto, or force e.g. to "en_US.UTF-8"

import time
from datetime import datetime, timedelta, date
import random

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


...


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Visualização (matplotlib, seaborn)

In [511]:
!python -m pip install matplotlib
!python -m pip install seaborn

import matplotlib.pyplot as plt
# import seaborn as sns

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Automação (selenium, schedule, BeautifulSoup)

In [512]:
# !python -m pip install selenium

# from selenium import webdriver
# # from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

In [513]:
# !python -m pip install schedule
# import schedule

## Web (BeautifulSoup, requests)

In [514]:
!python -m pip install BeautifulSoup
from bs4 import BeautifulSoup

Defaulting to user installation because normal site-packages is not writeable
  Using cached BeautifulSoup-3.2.2.tar.gz (32 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [7 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\ricardopeloi\AppData\Local\Temp\pip-install-qlk3911o\beautifulsoup_87c738a1c19344df9d063c2be1b9b79a\setup.py", line 3
          "You're trying to run a very old release of Beautiful Soup under Python 3. This will not work."<>"Please use Beautiful Soup 4, available through the pip package 'beautifulsoup4'."
                                                                                                         ^^
      SyntaxError: invalid syntax
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This i

In [515]:
!python -m pip install requests
import requests

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Funções

## Leitura de arquivos com ID no Drive

In [516]:
def le_arquivos_drive(id_arquivo):
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    !python -m pip install PyDrive# &> /dev/null

    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    # id_arquivo = "1sgkZW8mQCGdG_4DicIXFTCn3d-RBoRIk" # Altere o token dos dados aqui

    download = drive.CreateFile({'id': id_arquivo})

    download.GetContentFile('file.csv')
    dados  = pd.read_csv("file.csv")

    return dados
    # dados.head()

## Leitura de arquivos em uma pasta

In [517]:
def empilha_arquivos_pasta(
    caminho_pasta,
    formato, 
    sep = ",",
    imprimir_nomes_arquivos = True, 
    adicionar_arquivo_base = True,
    imprimir_check = True,
    remover_duplicatas = False,
    redefinir_index = False,
    ):
    
    bd = pd.DataFrame()
    check = 0
    arquivos_nao_lidos = []

    for filename in os.listdir(caminho_pasta):
        if (filename.endswith("." + formato.lower())) | (formato == ""):

            if imprimir_nomes_arquivos == True:
                print("Lendo arquivo " + filename)

            try:
                if formato.lower() == "csv":
                    try:
                        try:
                            novoArquivo = pd.read_csv(caminho_pasta + "\\" + filename, sep = sep, encoding = "utf-8", engine = "python")
                        except:
                            novoArquivo = pd.read_csv(caminho_pasta + "\\" + filename, encoding = "utf-8", engine = "python")
                    except:
                        try:
                            novoArquivo = pd.read_csv(caminho_pasta + "\\" + filename, sep = sep, encoding='iso-8859-1', engine = "python")
                        except:
                            novoArquivo = pd.read_csv(caminho_pasta + "\\" + filename, encoding='iso-8859-1', engine = "python")
                            
                elif formato.lower() == "xlsx":
                    novoArquivo = pd.read_excel(caminho_pasta)

                if adicionar_arquivo_base == True:
                    novoArquivo["Arquivo"] = filename

                check = check + len(novoArquivo)
                bd = pd.concat([bd, novoArquivo])
                print("Leitura do arquivo " + filename + " concluída")
            
            except:
                arquivos_nao_lidos.append(filename)
            
            continue
        else:
            continue

    if imprimir_check == True:
        if (len(bd) == check):
            check = "ok"
        else:
            check = "nok"
        print("Check: " + check + ", " + str(len(bd))) 

    if remover_duplicatas == True:
        bd = bd.drop_duplicates()

    if redefinir_index == True:
        bd = bd.reset_index().drop("index", axis = 1)
        
    return [bd, arquivos_nao_lidos]

## Elimina colunas NA

In [518]:
def elimina_colunas_NA(
    base,
    imprime_colunas_vazias = False,
    imprime_colunas_completas = False,
    imprime_colunas_parciais = False,
    remove_da_base = True,
    retorna_parciais = False
    ):

    tamanho_da_base = len(base)

    colunas_vazias = []
    colunas_completas = []
    colunas_parciais = []

    for coluna in base.columns:
        tamanho_da_coluna = len(base[base[coluna].isna()])
        if tamanho_da_coluna == tamanho_da_base:
            # print(coluna)
            colunas_vazias.append(coluna)
        elif tamanho_da_coluna == 0:
            colunas_completas.append(coluna)
        else:
            colunas_parciais.append(coluna)

    if imprime_colunas_vazias == True:
        if len(colunas_vazias) == 0:
            print("Não há colunas NA")
        else:
            print("Colunas vazias: " + str(colunas_vazias))
            
    if imprime_colunas_completas == True:
        if len(colunas_completas) == 0:
            print("Não há colunas completas")
        else:
            print("Colunas completas: " + str(colunas_completas))
    
    if imprime_colunas_parciais == True:
        if len(colunas_parciais) == 0:
            print("Não há colunas parciais")
        else:
            print("Colunas parciais: " + str(colunas_parciais))

    if remove_da_base == True:
        base = base.drop(colunas_vazias, axis = 1)
    
    if retorna_parciais == True:
        return [base, colunas_parciais]
    else:
        return base

## Adiciona campo agrupado

In [519]:
def adiciona_campo_agrupado(
    bd_limpa, bd_unificada, 
    campos_identificadores, 
    novo_campo, 
    imprime_campos = True,
    imprime_checks = True
    ):
    
    novo_campos_identificadores = campos_identificadores.copy()
    novo_campos_identificadores.append(novo_campo)
    if imprime_campos == True:
        print(str(novo_campos_identificadores))

    bd_limpa_com_campos = bd_limpa[novo_campos_identificadores] \
        .groupby(novo_campos_identificadores).first().reset_index()

    bd_limpa_com_campos = bd_limpa_com_campos.groupby(campos_identificadores).last()

    if novo_campo in bd_unificada.columns:
        bd_unificada_novo = bd_unificada.drop(novo_campo, axis = 1).join(bd_limpa_com_campos, how = "outer")
    else:
        bd_unificada_novo = bd_unificada.join(bd_limpa_com_campos, how = "outer")

    if imprime_checks == True:
        print(novo_campo + " vazios: " + str(len(bd_unificada_novo[bd_unificada_novo[novo_campo].isna()])))
        print(novo_campo + " não vazios: " + str(len(bd_unificada_novo[~bd_unificada_novo[novo_campo].isna()])))

    return bd_unificada_novo

## Limpa memória

In [520]:
def retrieve_name(var):
    import inspect

    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    
    return [var_name for var_name, var_val in callers_local_vars if var_val is var][0]

def limpa(ponteiro):
    try:
        del(ponteiro)
        import gc
        gc.collect()
    except:
        print("Não foi possível limpar "+ retrieve_name(ponteiro))

## Completa dígitos (padronizar chaves, CPF, etc)

In [521]:
def completa_digitos(texto, qtd):
    quantidade_zeros = qtd - len(texto)

    contador = quantidade_zeros
    while contador > 0:
        texto = "0" + texto
        contador = contador - 1

    # print(len(texto))

    return texto

## Exemplo de uso
# df["coluna_nova"] = df.apply(lambda x: completa_digitos(x["coluna_referencia"], 2), axis = 1)

## Consolida primeiro campo não NA entre duas opções

In [522]:
def consolida_primeiro_campo_existente(campoA, campoB):
    try:
        import numpy as np

        if np.isnan(campoA):
            return campoB
        else:
            return campoA
    except:
        if pd.isnull(campoA):
            return campoB
        else:
            return campoA

## Agrupar cada coluna 

In [523]:
def agrupamento_cada_coluna(
    bd, 
    coluna_completa, 
    colunas_ignoradas = [], 
    ascending = False,
    imprime_tabelas = True,
    ):
    
    from IPython.display import display

    campos_com_erro = []
    dict_campos = {}

    colunas = bd.columns.drop(coluna_completa)

    if len(colunas_ignoradas) > 0:
        colunas = colunas.drop(colunas_ignoradas)
    
    for coluna in colunas:
        try: 
            temp_coluna = bd.fillna("(vazio)").groupby(coluna).count()[[coluna_completa]].rename(columns = {coluna_completa: "Quantidade"}).sort_values("Quantidade", ascending = ascending)

            if ascending == False:
                temp_coluna["%"] = temp_coluna["Quantidade"]/temp_coluna["Quantidade"].sum()
                temp_coluna["% acumulado"] = temp_coluna["%"].cumsum()
            
            if imprime_tabelas == True:
                display(temp_coluna)
                
            dict_campos[coluna] = temp_coluna
            limpa(temp_coluna)
        
        except:
            campos_com_erro.append(coluna)
            #print("Campo " + coluna + " deu erro =/")
    
    return [campos_com_erro, dict_campos]

## Exemplos de repetições na BD

In [524]:
def exemplos_repeticoes_na_bd(
    bd_filtro_linhas, 
    campos_filtro_colunas, 
    coluna_completa, 
    ascending = False
    ):
    
    print("A tabela possui " + str(len(bd_filtro_linhas[campos_filtro_colunas])) + " linhas na base vezes na base")
    
    bd_agrupada = bd_filtro_linhas[campos_filtro_colunas+[coluna_completa]] \
        .groupby(campos_filtro_colunas).count() \
        .rename(columns = {coluna_completa: "Quantidade"}) \
        .sort_values("Quantidade", ascending = ascending)

    return bd_agrupada

## Histograma padrão

In [525]:
def histograma_padrao(
  dados_x,
  titulo = np.nan,
  dados_indice = np.nan,
  tipo = "hist",
  grid = False,
  legend = True,
  linha_media = np.nan,
  qtd_bins = 10,
  espessura = 0.8,
  limite_x_min = np.nan, limite_x_max = np.nan,
  limite_y_min = np.nan, limite_y_max = np.nan,
  ):
  
  plt.figure(figsize = (20,8))

  if not(pd.isnull(titulo)):
      plt.title(titulo, fontsize = 16)

  if tipo == "hist":
      n, bins, edges = plt.hist(
          dados_x,
          bins = qtd_bins,
          rwidth = espessura
          )
      plt.xticks(bins)
      
  elif tipo == "bar":
      plt.bar(
          dados_indice,
          dados_x,
          width = espessura
          )

  
  
  if not(math.isnan(linha_media)):
      plt.axvline(x=linha_media, color = "black", label='Média')
  
  if legend == True:
      plt.legend()
  
  plt.grid(grid)

  if (not(math.isnan(limite_x_max))) & (not(math.isnan(limite_x_min))):
      # print(limite_x_min)
      # print(~math.isnan(limite_x_max))
      plt.xlim(limite_x_min, limite_x_max)
  
  if (not(math.isnan(limite_y_max))) & (not(math.isnan(limite_y_min))):
      plt.ylim(limite_y_min, limite_y_max)
  
  plt.show()

## Gráfico com rótulos

In [526]:
def grafico_com_rotulos(
    dados, coluna_x, coluna_y, 
    tipo_grafico = "bar", titulo_grafico = "", tamanho_grafico = (20, 8),
    fig = None, ax = None, 
    tamanho_fonte = 12, grossura_barra = 1,
    cor_x = "b", rotacao_x = 0, 
    formato_eixo_y = "{x:.2f}",
    formato_y = "{:.2f}", rotacao_y = 0, limite_min_y = None, limite_max_y = None,
    xytext_label_y = (0,10),
  ):
  
  # dados_plot = dados_cliente[dados_cliente["% Faturamento Acumulado"] < 0.40]

  if fig is None:
    fig = plt.figure(figsize = tamanho_grafico)
  
  plt.clf()
  plt.rcParams.update({'font.size': tamanho_fonte})


  ys = dados[coluna_y]
  xs = np.arange(len(dados[coluna_x]))

  if (tipo_grafico == "bar") | (tipo_grafico == "line"):
    if (tipo_grafico == "bar"):
      plt.bar(
          x = xs,
          height = ys,
          width = grossura_barra,
          label = coluna_y,
      )
    elif (tipo_grafico == "line"):
      plt.plot(
        xs,
        ys,
        cor_x + "o-",
        label = coluna_y,
      )

    # display(cor_x + "o-")

    ax1 = plt.gca()
    if (limite_min_y is not None) & (limite_max_y is not None):
      ax1.set_ylim([min(dados[coluna_y])*limite_min_y, max(dados[coluna_y])*limite_max_y])

    ax1.set_ylabel(coluna_y, color = cor_x)
    ax1.set_xlabel(coluna_x)

    for x,y in zip(xs,ys):
        label = formato_y.format(y)

        plt.annotate(label, # this is the text
                    (x,y), # these are the coordinates to position the label
                    textcoords="offset points", # how to position the text
                    xytext = xytext_label_y, # distance from text to points (x,y)
                    ha='center', # horizontal alignment can be left, right or center
                    color = cor_x,
                    rotation = rotacao_y)


    ax1.xaxis.set_tick_params(rotation = rotacao_x)
    ax1.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter(formato_eixo_y))

    # fig.legend()
    plt.xticks(xs, dados[coluna_x])

  elif tipo_grafico == "barh":
    plt.barh(
        y = xs,
        width = ys,
        height = grossura_barra,
        # label = coluna_y,
    )
    ax1 = plt.gca()
    ax1.set_yticks(xs, dados[coluna_x])

    ax1.set_ylabel(coluna_x)
    ax1.set_xlabel(coluna_y, color = cor_x)
    ax1.invert_yaxis()

    if (limite_min_y is not None) & (limite_max_y is not None):
      ax1.set_xlim([min(dados[coluna_y])*limite_min_y, max(dados[coluna_y])*limite_max_y])

    for x,y in zip(xs,ys):
      label = formato_y.format(y)

      plt.annotate(label, # this is the text
                (y, x), # these are the coordinates to position the label
                textcoords="offset points", # how to position the text
                xytext = (xytext_label_y[1], xytext_label_y[0]), # distance from text to points (x,y)
                ha='center', # horizontal alignment can be left, right or center
                color = cor_x,
                rotation = rotacao_y)

    ax1.xaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter(formato_eixo_y))

  plt.title(titulo_grafico)
  plt.show()

## Concatenação de colunas na tabela de parâmetros

In [527]:
def concat_parametros(
  lista_parametros,
  tabela,
):

  str_parametros = ""
  for parametro in lista_parametros:
    if parametro == lista_parametros[-1]:
      str_parametros = str_parametros + str(parametro)

    else:
      str_parametros = str(parametro) + " - " + str_parametros

  tabela[str_parametros] = ""
  for parametro in lista_parametros:
    if parametro == lista_parametros[-1]:
      tabela[str_parametros] = tabela[str_parametros] + tabela[parametro].astype(str)

    else:
      tabela[str_parametros] = tabela[parametro].astype(str) + " - " + tabela[str_parametros]
      # str_parametros = str_parametros + parametro
    

#   tabela_resultados

  return (str_parametros, tabela)

## De Para

In [528]:
def de_para(tabela_de_para, coluna_de):
    try:
        return tabela_de_para.loc[coluna_de][0]
    except:
        return np.nan

## Encaixa na faixa

In [529]:
def encaixa_na_faixa(faixa_minima, faixa_maxima, valor, tipo = "less-inclusive"):
    faixa_minima.sort()
    faixa_maxima.sort()

    tabela_personalizada = pd.DataFrame(index = faixa_minima, data = faixa_maxima) \
                                        .reset_index() \
                                        .rename(columns = {"index": "Faixa mínima", 0: "Faixa máxima"}).astype(float)

    tabela_personalizada["Faixa"] = tabela_personalizada["Faixa mínima"].astype(str).str.split(".").str[0] + \
                                     " a " + \
                                     tabela_personalizada["Faixa máxima"].astype(str).str.split(".").str[0]
    
    if tipo == "more-inclusive":
        
        for contador in range(len(tabela_personalizada)):
            if (valor > tabela_personalizada.iloc[contador]["Faixa mínima"]) & (valor <= tabela_personalizada.iloc[contador]["Faixa máxima"]):
                # display(tabela_personalizada.iloc[contador])
                return tabela_personalizada.iloc[contador]["Faixa"]
        
        return "Fora das faixas"

    if tipo == "inclusive":
        
        for contador in range(len(tabela_personalizada)):
            if (valor >= tabela_personalizada.iloc[contador]["Faixa mínima"]) & (valor <= tabela_personalizada.iloc[contador]["Faixa máxima"]):
                # display(tabela_personalizada.iloc[contador])
                return tabela_personalizada.iloc[contador]["Faixa"]
        
        return "Fora das faixas"

    # if tipo == "exclusive": \\ mutuamente exclusivo (não tem igual)

    else:
    # if tipo == "less-inclusive":

        for contador in range(len(tabela_personalizada)):
            if (valor >= tabela_personalizada.iloc[contador]["Faixa mínima"]) & (valor < tabela_personalizada.iloc[contador]["Faixa máxima"]):
                # display(tabela_personalizada.iloc[contador])
                return tabela_personalizada.iloc[contador]["Faixa"]
        
        return "Fora das faixas"


# lista_idade_minima = [float("-inf"), 17, 25, 31, 41, 51]
# lista_idade_maxima = [16, 24, 30, 40, 50, float("inf")]

# encaixa_na_faixa(lista_idade_minima, lista_idade_maxima, 0)

## Amostra da base

In [530]:
def amostra_da_base(bd, qtd_por_grupo, taxa_de_conversao, random_state = 1):
    quantidade = min(int(round(qtd_por_grupo/taxa_de_conversao, 0)), len(bd))
    
    if random_state == False:
        return bd.sample(n = quantidade)
    else:
        return bd.sample(n = quantidade, random_state = random_state)

## Análise exploratória básica

In [531]:
def analise_exploratoria(
    bd,
    imprime_todas_colunas = False,
    imprime_info_colunas = True,
    imprime_colunas_vazias = False,
    imprime_colunas_completas = False,
    imprime_colunas_parciais = False,
    remove_da_base = True,
    retorna_parciais = False,
    detalhar_colunas_parciais = True,
    colunas_ignoradas = []
    ):

    # COMEÇANDO PELAS COLUNAS DISPONÍVEIS E INFO
    if imprime_todas_colunas == True:
        display(bd.columns)
    
    if imprime_info_colunas == True:
        for i in range(int(np.ceil(len(bd.columns)/20))):
            display(bd.iloc[:, (i*20):min((i+1)*20, len(bd.columns))].info())

    # DETALHAMENTO DE QUAIS COLUNAS SÃO NA OU PARCIAIS
    if retorna_parciais == True:
        [bd_semNA, colunas_parciais] = elimina_colunas_NA(
            bd,
            imprime_colunas_vazias = imprime_colunas_vazias,
            imprime_colunas_completas = imprime_colunas_completas,
            imprime_colunas_parciais = imprime_colunas_parciais,
            remove_da_base = remove_da_base,
            retorna_parciais = retorna_parciais
        )

        if detalhar_colunas_parciais == True:
            for coluna in colunas_parciais:
                print("# " + coluna + ": " + str(len(colunas_parciais[colunas_parciais[coluna].isna()])))
    else:
        colunas_parciais = []

        bd_semNA = elimina_colunas_NA(
            bd,
            imprime_colunas_vazias = imprime_colunas_vazias,
            imprime_colunas_completas = imprime_colunas_completas,
            imprime_colunas_parciais = imprime_colunas_parciais,
            remove_da_base = remove_da_base,
            retorna_parciais = retorna_parciais
        )

    [campos_com_erro, colunas_agrupadas] = agrupamento_cada_coluna(
        bd.reset_index(), 
        coluna_completa = bd_semNA.drop(colunas_parciais, axis = 1).reset_index().columns[0],
        colunas_ignoradas = colunas_ignoradas
    )
    
    if retorna_parciais == True:
        return [bd_semNA, colunas_parciais, colunas_agrupadas, campos_com_erro]
    else:
        return [bd_semNA, colunas_agrupadas, campos_com_erro]

## GridSearch

In [532]:
def treina_modelo_grid(
  x_train,
  y_train,
  modelo,
  param_grid,
  cv = 10,
  scoring = "neg_mean_absolute_error",
  tipo = "grid",
  random_state_seed = 1082141,
  n_iter = 50
):
  from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
  
  if tipo == "grid":
      grid = GridSearchCV(
          modelo, 
          param_grid, 
          cv = cv, 
          scoring = scoring, 
          return_train_score = True,
      )

  elif tipo == "randomized":
      # print("Busca randomizada em " + str(n_iter) + " combinações de parâmetros.")
      grid = RandomizedSearchCV(
          modelo, 
          param_grid, 
          cv = cv, 
          scoring = scoring, 
          return_train_score = True,
          random_state = random_state_seed,
          n_iter = n_iter,
      )

  grid.fit(x_train, y_train)

  return grid

### Definimos range de hiperparâmetros
# n_neighbors_range = range(10, 20)
# metric_range = ["cityblock", "cosine", "euclidean", "haversine", "l1", "l2", "manhattan", "nan_euclidean", "minkowski"]
# algorithm_range = ["auto", "ball_tree", "kd_tree", "brute"]

# param_grid = dict(algorithm = algorithm_range)
# # display(param_grid) 

# grid_melhor_algorithm = treina_modelo_grid(
#     dados_treino_X,
#     dados_treino_y,
#     KNeighborsClassifier(n_neighbors = 12, metric = "euclidean"),
#     param_grid,
#     cv = 5,
#     scoring = "accuracy",
#     tipo = "grid",
#     random_state_seed = numero_aleatorio,
#     # n_iter = 50
# )

# # print(grid.best_score_)
# # grid.best_estimator_
# tabela_resultados = pd.concat([
#   pd.json_normalize(pd.DataFrame(grid_melhor_algorithm.cv_results_)["params"]),
#   pd.DataFrame(grid_melhor_algorithm.cv_results_)[["rank_test_score", "mean_test_score"]]
# ], axis = 1).set_index("rank_test_score").sort_index()


# plt.figure(figsize = (15,8))
# display(tabela_resultados)
# plt.bar(x = tabela_resultados["algorithm"], height = tabela_resultados["mean_test_score"]);

# Brasileirão

## Leitura (site CBF)

In [707]:
# Make a request to the webpage
brasileirao_url = 'https://www.cbf.com.br/futebol-brasileiro'
brasileirao_response = requests.get(brasileirao_url)

# Create a BeautifulSoup object with the HTML content
brasileirao_soup = BeautifulSoup(brasileirao_response.content, 'html.parser')

# soup

### Data e numeração

In [708]:
# Find all div elements with the class 'clearfix'
brasileirao_jogos_lista_datas = brasileirao_soup.find_all('span', class_='partida-desc text-1 color-lightgray p-b-15 block uppercase text-center')

brasileirao_jogos_lista_datas[0]

<span class="partida-desc text-1 color-lightgray p-b-15 block uppercase text-center">
                              Sáb, 15/04/2023 16:00
                            -  Jogo: 10 
                          </span>

In [709]:
# Initialize a list to store the extracted data
brasileirao_lista_datas_numeracao = []

# Iterate over the div elements
for jogo in brasileirao_jogos_lista_datas:
    # Extract the specific fields from the div and create a dictionary
    jogo_texto = jogo.text.strip()
    # print(jogo.text)

    # Add the dictionary to the data list
    brasileirao_lista_datas_numeracao.append({
        'data_jogo': jogo_texto,
    })

brasileirao_lista_datas_numeracao

[{'data_jogo': 'Sáb, 15/04/2023 16:00\r\n                            -  Jogo: 10'},
 {'data_jogo': 'Sáb, 15/04/2023 16:00\r\n                            -  Jogo: 3'},
 {'data_jogo': 'Sáb, 15/04/2023 18:30\r\n                            -  Jogo: 5'},
 {'data_jogo': 'Sáb, 15/04/2023 18:30\r\n                            -  Jogo: 8'},
 {'data_jogo': 'Sáb, 15/04/2023 18:30\r\n                            -  Jogo: 9'},
 {'data_jogo': 'Sáb, 15/04/2023 18:30\r\n                            -  Jogo: 2'},
 {'data_jogo': 'Sáb, 15/04/2023 21:00\r\n                            -  Jogo: 6'},
 {'data_jogo': 'Dom, 16/04/2023 16:00\r\n                            -  Jogo: 4'},
 {'data_jogo': 'Dom, 16/04/2023 16:00\r\n                            -  Jogo: 1'},
 {'data_jogo': 'Dom, 16/04/2023 18:30\r\n                            -  Jogo: 7 \r\n                              \r\n                  1 alteração'},
 {'data_jogo': 'Sáb, 22/04/2023 16:00\r\n                            -  Jogo: 11'},
 {'data_jogo': 'S

### Local

In [710]:
# Find all div elements with the class 'clearfix'
brasileirao_jogos_lista_local = soup.find_all('span', class_='partida-desc text-1 color-lightgray block uppercase text-center')

brasileirao_jogos_lista_local[0]

<span class="partida-desc text-1 color-lightgray block uppercase text-center">
              Independência - Belo Horizonte - MG<br/>
<a class="btn btn-xs btn-success m-t-5" href="https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a/2023/10?ref=botao" rel="noopener" target="_blank">Como foi o jogo</a>
</span>

In [711]:
# Initialize a list to store the extracted data
brasileirao_lista_local = []

# Iterate over the div elements
for jogo in brasileirao_jogos_lista_local:
    # Extract the specific fields from the div and create a dictionary
    local_jogo = jogo.text.strip()
    # print(jogo.text)

    # Add the dictionary to the data list
    brasileirao_lista_local.append({
        'local_jogo': local_jogo,
    })

brasileirao_lista_local

[{'local_jogo': 'Independência - Belo Horizonte - MG\nComo foi o jogo'},
 {'local_jogo': 'Allianz Parque - Sao Paulo - SP\nComo foi o jogo'},
 {'local_jogo': 'Nabi Abi Chedid - Braganca Paulista - SP\nComo foi o jogo'},
 {'local_jogo': 'Ligga Arena - Curitiba - PR\nComo foi o jogo'},
 {'local_jogo': 'Arena Castelão - Fortaleza - CE\nComo foi o jogo'},
 {'local_jogo': 'Nilton Santos - Rio de Janeiro - RJ\nComo foi o jogo'},
 {'local_jogo': 'Mineirão - Belo Horizonte - MG\nComo foi o jogo'},
 {'local_jogo': 'Neo Química Arena - Sao Paulo - SP\nComo foi o jogo'},
 {'local_jogo': 'Maracanã - Rio de Janeiro - RJ\nComo foi o jogo'},
 {'local_jogo': 'Alfredo Jaconi - Caxias do Sul - RS\nComo foi o jogo'},
 {'local_jogo': 'Maracanã - Rio de Janeiro - RJ\nComo foi o jogo'},
 {'local_jogo': 'Arena Pantanal - Cuiaba - MT\nComo foi o jogo'},
 {'local_jogo': 'Morumbi - Sao Paulo - SP\nComo foi o jogo'},
 {'local_jogo': 'Independência - Belo Horizonte - MG\nComo foi o jogo'},
 {'local_jogo': 'Beira-

### Demais detalhes do jogo

In [712]:
brasileirao_jogos_lista_timeA_timeB_resultado = soup.find_all('div', class_='clearfix')
brasileirao_jogos_lista_timeA_timeB_resultado[0]

<div class="clearfix">
<a class="no-underline" href="https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a/2023/10?ref=linha" rel="noopener" target="_blank">
<div class="time pull-left">
<span class="time-sigla">AME</span>
<img alt="América Fc Saf - MG" class="icon escudo x45 pull-right" onerror="this.src='https://conteudo.cbf.com.br/cdn/imagens/escudos/empty.jpg'" src="https://conteudo.cbf.com.br/cdn/imagens/escudos/59897mg.jpg?v=2023062417" title="América Fc Saf - MG"/>
</div>
<div class="time pull-right">
<span class="time-sigla">FLU</span>
<img alt="Fluminense - RJ" class="icon escudo x45 pull-left" onerror="this.src='https://conteudo.cbf.com.br/cdn/imagens/escudos/empty.jpg'" src="https://conteudo.cbf.com.br/cdn/imagens/escudos/00009rj.jpg?v=2023062417" title="Fluminense - RJ"/>
</div>
<strong class="partida-horario center-block">
<span class="bg-blue color-white label-2">0 x 3</span>
</strong>
</a>
</div>

In [713]:
# Initialize a list to store the extracted data
brasileirao_lista_timeA_timeB_resultado = []

# Iterate over the div elements
for jogo in brasileirao_jogos_lista_timeA_timeB_resultado:
    # Extract the specific fields from the div and create a dictionary
    link_jogo = jogo.find("a", class_="no-underline").get('href')
    timeA_sigla = jogo.find('div', class_='time pull-left').find('span', class_='time-sigla').text.strip()
    timeA_nome_alt = jogo.find('div', class_='time pull-left').find('img').get("alt")
    timeB_sigla = jogo.find('div', class_='time pull-right').find('span', class_='time-sigla').text.strip()
    timeB_nome_alt = jogo.find('div', class_='time pull-right').find('img').get("alt")
    try:
        resultado = jogo.find('span', class_='bg-blue color-white label-2').text.strip()
    except:
        resultado = "Jogo ainda não ocorreu"

    
    # Add the dictionary to the data list
    brasileirao_lista_timeA_timeB_resultado.append({
        'link_jogo': link_jogo,
        'timeA_sigla': timeA_sigla,
        'timeA_nome': timeA_nome_alt,
        'timeB_sigla': timeB_sigla,
        'timeB_nome': timeB_nome_alt,
        'resultado': resultado,
        'fase': np.nan,
        'ida ou volta': "Única",
        'torneio': "Brasileirão",
    })


brasileirao_lista_timeA_timeB_resultado

[{'link_jogo': 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a/2023/10?ref=linha',
  'timeA_sigla': 'AME',
  'timeA_nome': 'América Fc Saf - MG',
  'timeB_sigla': 'FLU',
  'timeB_nome': 'Fluminense - RJ',
  'resultado': '0 x 3',
  'fase': nan,
  'ida ou volta': 'Única',
  'torneio': 'Brasileirão'},
 {'link_jogo': 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a/2023/3?ref=linha',
  'timeA_sigla': 'PAL',
  'timeA_nome': 'Palmeiras - SP',
  'timeB_sigla': 'CUI',
  'timeB_nome': 'Cuiabá Saf - MT',
  'resultado': '2 x 1',
  'fase': nan,
  'ida ou volta': 'Única',
  'torneio': 'Brasileirão'},
 {'link_jogo': 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a/2023/5?ref=linha',
  'timeA_sigla': 'RED',
  'timeA_nome': 'Red Bull Bragantino - SP',
  'timeB_sigla': 'BAH',
  'timeB_nome': 'Bahia - BA',
  'resultado': '2 x 1',
  'fase': nan,
  'ida ou volta': 'Única',
  'torneio': 'Brasileirão

In [714]:
# # Convert the data to a pandas DataFrame
brasileirao_base_timeA_timeB_resultado = pd.DataFrame(brasileirao_lista_timeA_timeB_resultado)
brasileirao_base_datas_numeracao = pd.DataFrame(brasileirao_lista_datas_numeracao)
brasileirao_base_local = pd.DataFrame(brasileirao_lista_local)

# # Print or save the JSON and DataFrame
# print(json_data)
brasileirao_base_timeA_timeB_resultado_datas_numeracao = brasileirao_base_timeA_timeB_resultado.join(brasileirao_base_datas_numeracao).join(brasileirao_base_local)

brasileirao_base_timeA_timeB_resultado_datas_numeracao

,link_jogo,timeA_sigla,timeA_nome,timeB_sigla,timeB_nome,resultado,fase,ida ou volta,torneio,data_jogo,local_jogo
0,https://www.cbf.com.br/futebol-brasileiro/comp...,AME,América Fc Saf - MG,FLU,Fluminense - RJ,0 x 3,NaN,Única,Brasileirão,"Sáb, 15/04/2023 16:00\r\n ...",Independência - Belo Horizonte - MG\nComo foi ...
1,https://www.cbf.com.br/futebol-brasileiro/comp...,PAL,Palmeiras - SP,CUI,Cuiabá Saf - MT,2 x 1,NaN,Única,Brasileirão,"Sáb, 15/04/2023 16:00\r\n ...",Allianz Parque - Sao Paulo - SP\nComo foi o jogo
2,https://www.cbf.com.br/futebol-brasileiro/comp...,RED,Red Bull Bragantino - SP,BAH,Bahia - BA,2 x 1,NaN,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",Nabi Abi Chedid - Braganca Paulista - SP\nComo...
3,https://www.cbf.com.br/futebol-brasileiro/comp...,ATH,Athletico Paranaense - PR,GOI,Goiás - GO,2 x 0,NaN,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",Ligga Arena - Curitiba - PR\nComo foi o jogo
4,https://www.cbf.com.br/futebol-brasileiro/comp...,FOR,Fortaleza - CE,INT,Internacional - RS,1 x 1,NaN,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",Arena Castelão - Fortaleza - CE\nComo foi o jogo
...,...,...,...,...,...,...,...,...,...,...,...
375,https://www.cbf.com.br/futebol-brasileiro/comp...,GOI,Goiás - GO,AME,América Fc Saf - MG,Jogo ainda não ocorreu,NaN,Única,Brasileirão,A definir - Jogo: 375,A definir\nDetalhes do jogo
376,https://www.cbf.com.br/futebol-brasileiro/comp...,SAN,Santos - SP,FOR,Fortaleza - CE,Jogo ainda não ocorreu,NaN,Única,Brasileirão,A definir - Jogo: 374,A definir\nDetalhes do jogo
377,https://www.cbf.com.br/futebol-brasileiro/comp...,SAO,São Paulo - SP,FLA,Flamengo - RJ,Jogo ainda não ocorreu,NaN,Única,Brasileirão,A definir - Jogo: 373,A definir\nDetalhes do jogo
378,https://www.cbf.com.br/futebol-brasileiro/comp...,VAS,Vasco da Gama S.a.f. - RJ,RED,Red Bull Bragantino - SP,Jogo ainda não ocorreu,NaN,Única,Brasileirão,A definir - Jogo: 372,A definir\nDetalhes do jogo


# Copa do Brasil

## Leitura (site CBF)

### 1ª fase

In [760]:
# Make a request to the webpage
copabrasil_url_1fase = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023?phase=1663'
copabrasil_response_1fase = requests.get(copabrasil_url_1fase)

# Create a BeautifulSoup object with the HTML content
copabrasil_soup_1fase = BeautifulSoup(copabrasil_response_1fase.content, 'html.parser')

# copabrasil_soup_1fase

In [761]:
copabrasil_grupos_1fase = copabrasil_soup_1fase.find_all('div', class_='box p-t-25 p-b-15')

# len(copabrasil_grupos_1fase)
copabrasil_grupos_1fase[1]

<div class="box p-t-25 p-b-15">
<div class="block">
<div class="text-1 m-b-10 text-center uppercase">
                    Qua, 22/02/2023 16:00
            - GRUPO 26 |  Jogo: 26 
              - <span class="color-red" style="font-size: 9px">
          1 alteração
        </span>
</div>
<a class="no-underline" href="https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/26?ref=linha" rel="noopener" target="_blank">
<div class="clearfix">
<div class="col-xs-5 p-t-10 nopadding">
<img alt="Falcon Futebol Clube - SE" class="icon escudo x54 pull-right" onerror="this.src='https://conteudo.cbf.com.br/cdn/imagens/escudos/empty.jpg'" src="https://conteudo.cbf.com.br/cdn/imagens/escudos/58978se.jpg?v=2023062419" title="Falcon Futebol Clube - SE"/>
<b class="text-2 pull-right p-t-15 p-r-10 hidden-lg hidden-md">FAL</b>
<div class="text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm">Falcon Futebol Clube - SE</div>
</div>
<div class="col-xs-2 color-blue text-center nopadd

In [762]:
# Initialize a list to store the extracted data
copabrasil_lista_timeA_timeB_resultado = []

# Iterate over the div elements
for jogo in copabrasil_grupos_1fase:

    data_jogo = jogo.find("div", class_="text-1 m-b-10 text-center uppercase").text.strip()
    link_jogo = jogo.find("a", class_="no-underline").get("href")
    local_jogo = jogo.find("div", class_="text-1 m-t-10 text-center uppercase").text.strip()
    timeA_sigla = jogo.find('b', class_='text-2 pull-right p-t-15 p-r-10 hidden-lg hidden-md').text.strip()
    timeA_nome = jogo.find('div', class_='text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm').text.strip()
    timeB_sigla = jogo.find('b', class_='text-2 pull-left p-t-15 p-l-10 hidden-lg hidden-md').text.strip()
    timeB_nome = jogo.find('div', class_='text-2 pull-left p-t-15 p-l-20 hidden-xs hidden-sm').text.strip()
    try:
        resultado = jogo.find('div', class_='text-3 p-t-15').text.strip()
    except:
        resultado = "Jogo ainda não ocorreu"
    if resultado == "x":
        resultado = "Jogo ainda não ocorreu"
    
    # Add the dictionary to the data list
    copabrasil_lista_timeA_timeB_resultado.append({
        'data_jogo': data_jogo,
        'link_jogo': link_jogo,
        'local_jogo': local_jogo,
        'timeA_sigla': timeA_sigla,
        'timeA_nome': timeA_nome,
        'timeB_sigla': timeB_sigla,
        'timeB_nome': timeB_nome,
        'resultado': resultado,
        'fase': "1ª Fase",
        'ida ou volta': "Única",
        'torneio': "Copa do Brasil",
    })


copabrasil_lista_timeA_timeB_resultado

[{'data_jogo': 'Ter, 21/02/2023 19:00\r\n            - GRUPO 19 |  Jogo: 19 \r\n              - \r\n          1 alteração',
  'link_jogo': 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/19?ref=linha',
  'local_jogo': 'Hercílio Luz - Itajai - SC\nComo foi o jogo',
  'timeA_sigla': 'MAR',
  'timeA_nome': 'Marcílio Dias - SC',
  'timeB_sigla': 'CHA',
  'timeB_nome': 'Chapecoense - SC',
  'resultado': '1 x 0',
  'fase': '1ª Fase',
  'ida ou volta': 'Única',
  'torneio': 'Copa do Brasil'},
 {'data_jogo': 'Qua, 22/02/2023 16:00\r\n            - GRUPO 26 |  Jogo: 26 \r\n              - \r\n          1 alteração',
  'link_jogo': 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/26?ref=linha',
  'local_jogo': 'Batistão - Aracaju - SE\nComo foi o jogo',
  'timeA_sigla': 'FAL',
  'timeA_nome': 'Falcon Futebol Clube - SE',
  'timeB_sigla': 'VOL',
  'timeB_nome': 'Volta Redonda - RJ',
  'resultado': '1 x 3',
  'fase': '1ª Fase',
  '

In [763]:
# # Convert the data to a pandas DataFrame
copabrasil_base_lista_timeA_timeB_resultado_1fase = pd.DataFrame(copabrasil_lista_timeA_timeB_resultado)

copabrasil_base_lista_timeA_timeB_resultado_1fase

,data_jogo,link_jogo,local_jogo,timeA_sigla,timeA_nome,timeB_sigla,timeB_nome,resultado,fase,ida ou volta,torneio
0,"Ter, 21/02/2023 19:00\r\n - GRUPO 1...",https://www.cbf.com.br/futebol-brasileiro/comp...,Hercílio Luz - Itajai - SC\nComo foi o jogo,MAR,Marcílio Dias - SC,CHA,Chapecoense - SC,1 x 0,1ª Fase,Única,Copa do Brasil
1,"Qua, 22/02/2023 16:00\r\n - GRUPO 2...",https://www.cbf.com.br/futebol-brasileiro/comp...,Batistão - Aracaju - SE\nComo foi o jogo,FAL,Falcon Futebol Clube - SE,VOL,Volta Redonda - RJ,1 x 3,1ª Fase,Única,Copa do Brasil
2,"Qua, 22/02/2023 17:00\r\n - GRUPO 4...",https://www.cbf.com.br/futebol-brasileiro/comp...,Valdir Doilho Wons - Nova Mutum - MT\nComo foi...,MUT,Mutum Esporte Clube - MT,LON,Londrina - PR,4 x 2,1ª Fase,Única,Copa do Brasil
3,"Qua, 22/02/2023 19:00\r\n - GRUPO 3...",https://www.cbf.com.br/futebol-brasileiro/comp...,Canarinho - Boa Vista - RR\nComo foi o jogo,SAO,São Raimundo - RR,CUI,Cuiabá Saf - MT,4 x 3,1ª Fase,Única,Copa do Brasil
4,"Qua, 22/02/2023 19:00\r\n - GRUPO 3...",https://www.cbf.com.br/futebol-brasileiro/comp...,João Ronaldo - Pacajus - CE\nComo foi o jogo,CAU,Caucaia - CE,TOM,Tombense - MG,0 x 0,1ª Fase,Única,Copa do Brasil
5,"Qua, 22/02/2023 20:00\r\n - GRUPO 1...",https://www.cbf.com.br/futebol-brasileiro/comp...,Bento de Abreu - Marilia - SP\nComo foi o jogo,MAR,Marilia - SP,BRU,Brusque - SC,0 x 3,1ª Fase,Única,Copa do Brasil
6,"Qua, 22/02/2023 21:00\r\n - GRUPO 3...",https://www.cbf.com.br/futebol-brasileiro/comp...,Florestão - Rio Branco - AC\nComo foi o jogo,SAO,Sao Francisco - AC,YPI,Ypiranga - RS,1 x 1,1ª Fase,Única,Copa do Brasil
7,"Qui, 23/02/2023 19:00\r\n - GRUPO 2...",https://www.cbf.com.br/futebol-brasileiro/comp...,Florestão - Rio Branco - AC\nComo foi o jogo,HUM,Humaitá - AC,COR,Coritiba - PR,0 x 3,1ª Fase,Única,Copa do Brasil
8,"Qui, 23/02/2023 19:00\r\n - GRUPO 1...",https://www.cbf.com.br/futebol-brasileiro/comp...,19 de Outubro - Ijui - RS\nComo foi o jogo,SAO,São Luiz - RS,JUV,Juventude - RS,1 x 0,1ª Fase,Única,Copa do Brasil
9,"Qui, 23/02/2023 20:00\r\n - GRUPO 3...",https://www.cbf.com.br/futebol-brasileiro/comp...,Boca do Jacaré / Serejão - Brasilia - DF\nComo...,CEI,Ceilândia - DF,SAN,Santos - SP,0 x 1,1ª Fase,Única,Copa do Brasil


### 2ª fase

In [764]:
# Make a request to the webpage
copabrasil_url_2fase = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023?phase=1672'
copabrasil_response_2fase = requests.get(copabrasil_url_2fase)

# Create a BeautifulSoup object with the HTML content
copabrasil_soup_2fase = BeautifulSoup(copabrasil_response_2fase.content, 'html.parser')

# copabrasil_soup_2fase

In [765]:
copabrasil_grupos_2fase = copabrasil_soup_2fase.find_all('div', class_='box p-t-25 p-b-15')

# len(copabrasil_grupos_2fase_ida)
copabrasil_grupos_2fase[1]

<div class="box p-t-25 p-b-15">
<div class="block">
<div class="text-1 m-b-10 text-center uppercase">
                    Qua, 08/03/2023 19:00
            - GRUPO 51 |  Jogo: 42 
          </div>
<a class="no-underline" href="https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/42?ref=linha" rel="noopener" target="_blank">
<div class="clearfix">
<div class="col-xs-5 p-t-10 nopadding">
<img alt="Camboriu Futebol Clube - SC" class="icon escudo x54 pull-right" onerror="this.src='https://conteudo.cbf.com.br/cdn/imagens/escudos/empty.jpg'" src="https://conteudo.cbf.com.br/cdn/imagens/escudos/00359sc.jpg?v=2023062419" title="Camboriu Futebol Clube - SC"/>
<b class="text-2 pull-right p-t-15 p-r-10 hidden-lg hidden-md">CAM</b>
<div class="text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm">Camboriu Futebol Clube - SC</div>
</div>
<div class="col-xs-2 color-blue text-center nopadding">
<div class="text-3 p-t-15">
<b>0 x 1</b>
</div>
</div>
<div class="col-xs-5 p-t-

In [766]:
# Initialize a list to store the extracted data
copabrasil_lista_timeA_timeB_resultado = []

# Iterate over the div elements
for jogo in copabrasil_grupos_2fase:

    data_jogo = jogo.find("div", class_="text-1 m-b-10 text-center uppercase").text.strip()
    link_jogo = jogo.find("a", class_="no-underline").get("href")
    local_jogo = jogo.find("div", class_="text-1 m-t-10 text-center uppercase").text.strip()
    timeA_sigla = jogo.find('b', class_='text-2 pull-right p-t-15 p-r-10 hidden-lg hidden-md').text.strip()
    timeA_nome = jogo.find('div', class_='text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm').text.strip()
    timeB_sigla = jogo.find('b', class_='text-2 pull-left p-t-15 p-l-10 hidden-lg hidden-md').text.strip()
    timeB_nome = jogo.find('div', class_='text-2 pull-left p-t-15 p-l-20 hidden-xs hidden-sm').text.strip()
    try:
        resultado = jogo.find('div', class_='text-3 p-t-15').text.strip()
    except:
        resultado = "Jogo ainda não ocorreu"
    if resultado == "x":
        resultado = "Jogo ainda não ocorreu"
    
    # Add the dictionary to the data list
    copabrasil_lista_timeA_timeB_resultado.append({
        'data_jogo': data_jogo,
        'link_jogo': link_jogo,
        'local_jogo': local_jogo,
        'timeA_sigla': timeA_sigla,
        'timeA_nome': timeA_nome,
        'timeB_sigla': timeB_sigla,
        'timeB_nome': timeB_nome,
        'resultado': resultado,
        'fase': "2ª Fase",
        'ida ou volta': "Única",
        'torneio': "Copa do Brasil",
    })


copabrasil_lista_timeA_timeB_resultado

[{'data_jogo': 'Ter, 07/03/2023 20:00\r\n            - GRUPO 42 |  Jogo: 41',
  'link_jogo': 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/41?ref=linha',
  'local_jogo': 'Bento Freitas - Pelotas - RS\nComo foi o jogo',
  'timeA_sigla': 'BRA',
  'timeA_nome': 'Brasil - RS',
  'timeB_sigla': 'PON',
  'timeB_nome': 'Ponte Preta - SP',
  'resultado': '2 x 0',
  'fase': '2ª Fase',
  'ida ou volta': 'Única',
  'torneio': 'Copa do Brasil'},
 {'data_jogo': 'Qua, 08/03/2023 19:00\r\n            - GRUPO 51 |  Jogo: 42',
  'link_jogo': 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/42?ref=linha',
  'local_jogo': 'Nações - Balneario Camboriu - SC\nComo foi o jogo',
  'timeA_sigla': 'CAM',
  'timeA_nome': 'Camboriu Futebol Clube - SC',
  'timeB_sigla': 'BAH',
  'timeB_nome': 'Bahia - BA',
  'resultado': '0 x 1',
  'fase': '2ª Fase',
  'ida ou volta': 'Única',
  'torneio': 'Copa do Brasil'},
 {'data_jogo': 'Qua, 08/03/2023 20:00\

In [767]:
# # Convert the data to a pandas DataFrame
copabrasil_base_lista_timeA_timeB_resultado_2fase = pd.DataFrame(copabrasil_lista_timeA_timeB_resultado)

copabrasil_base_lista_timeA_timeB_resultado_2fase

,data_jogo,link_jogo,local_jogo,timeA_sigla,timeA_nome,timeB_sigla,timeB_nome,resultado,fase,ida ou volta,torneio
0,"Ter, 07/03/2023 20:00\r\n - GRUPO 4...",https://www.cbf.com.br/futebol-brasileiro/comp...,Bento Freitas - Pelotas - RS\nComo foi o jogo,BRA,Brasil - RS,PON,Ponte Preta - SP,2 x 0,2ª Fase,Única,Copa do Brasil
1,"Qua, 08/03/2023 19:00\r\n - GRUPO 5...",https://www.cbf.com.br/futebol-brasileiro/comp...,Nações - Balneario Camboriu - SC\nComo foi o jogo,CAM,Camboriu Futebol Clube - SC,BAH,Bahia - BA,0 x 1,2ª Fase,Única,Copa do Brasil
2,"Qua, 08/03/2023 20:00\r\n - GRUPO 4...",https://www.cbf.com.br/futebol-brasileiro/comp...,Baenão - Belem - PA\nComo foi o jogo,REM,Remo - PA,SAO,São Luiz - RS,2 x 1,2ª Fase,Única,Copa do Brasil
3,"Qua, 08/03/2023 21:30\r\n - GRUPO 5...",https://www.cbf.com.br/futebol-brasileiro/comp...,Soares de Azevedo - Muriae - MG\nComo foi o jogo,TOM,Tombense - MG,RET,Retrô - PE,1 x 0,2ª Fase,Única,Copa do Brasil
4,"Qui, 09/03/2023 19:00\r\n - GRUPO 5...",https://www.cbf.com.br/futebol-brasileiro/comp...,Aflitos - Recife - PE\nComo foi o jogo,NAU,Náutico - PE,VIL,Vila Nova - GO,2 x 1,2ª Fase,Única,Copa do Brasil
5,"Qui, 09/03/2023 20:00\r\n - GRUPO 5...",https://www.cbf.com.br/futebol-brasileiro/comp...,Santa Cruz - Ribeirao Preto - SP\nComo foi o jogo,BOT,Botafogo - SP,SAO,São Raimundo - RR,3 x 1,2ª Fase,Única,Copa do Brasil
6,"Qui, 09/03/2023 21:30\r\n - GRUPO 5...",https://www.cbf.com.br/futebol-brasileiro/comp...,Vila Belmiro - Santos - SP\nComo foi o jogo,SAN,Santos - SP,IGU,Iguatu - CE,3 x 0,2ª Fase,Única,Copa do Brasil
7,"Ter, 14/03/2023 20:00\r\n - GRUPO 5...",https://www.cbf.com.br/futebol-brasileiro/comp...,Willie Davids - Maringa - PR\nComo foi o jogo,MAR,Maringá Futebol Clube S.a.f. - PR,MAR,Marcílio Dias - SC,2 x 0,2ª Fase,Única,Copa do Brasil
8,"Ter, 14/03/2023 20:00\r\n - GRUPO 5...",https://www.cbf.com.br/futebol-brasileiro/comp...,Couto Pereira - Curitiba - PR\nComo foi o jogo,COR,Coritiba - PR,CRI,Criciúma - SC,(6)1 x 1(5),2ª Fase,Única,Copa do Brasil
9,"Ter, 14/03/2023 21:30\r\n - GRUPO 4...",https://www.cbf.com.br/futebol-brasileiro/comp...,Independência - Belo Horizonte - MG\nComo foi ...,AME,América Fc Saf - MG,SAN,Santa Cruz - PE,1 x 0,2ª Fase,Única,Copa do Brasil


### 3ª fase

In [768]:
# Make a request to the webpage
copabrasil_url_3fase = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023?phase=1679'
copabrasil_response_3fase = requests.get(copabrasil_url_3fase)

# Create a BeautifulSoup object with the HTML content
copabrasil_soup_3fase = BeautifulSoup(copabrasil_response_3fase.content, 'html.parser')

# soup

#### Ida

In [769]:
copabrasil_grupos_3fase_ida = copabrasil_soup_3fase.find_all('div', class_='col-md-6 p-b-15 p-t-20 block border-right')

# len(copabrasil_grupos_oitavas_ida)
copabrasil_grupos_3fase_ida[1]

<div class="col-md-6 p-b-15 p-t-20 block border-right">
<div class="text-1 m-b-10 text-center uppercase">
                  Ter, 11/04/2023 19:00
                - GRUPO 61 |  Jogo: 61 
              </div>
<a class="no-underline" href="https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/61?ref=linha" rel="noopener" target="_blank">
<div class="clearfix">
<div class="col-xs-5 p-t-10 nopadding">
<img alt="Botafogo - SP" class="icon escudo x54 pull-right" onerror="this.src='https://conteudo.cbf.com.br/cdn/imagens/escudos/empty.jpg'" src="https://conteudo.cbf.com.br/cdn/imagens/escudos/56654sp.jpg?v=2023062419" title="Botafogo - SP"/>
<b class="text-2 pull-right p-t-15 p-r-10 hidden-lg">BOT</b>
<div class="text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm hidden-md">Botafogo - SP</div>
</div>
<div class="col-xs-2 color-blue text-center nopadding">
<div class="text-3 p-t-15">
<b>0 x 2</b>
</div>
</div>
<div class="col-xs-5 p-t-10 nopadding">
<img alt="Santos 

In [770]:
# Initialize a list to store the extracted data
copabrasil_lista_timeA_timeB_resultado = []

# Iterate over the div elements
for jogo in copabrasil_grupos_3fase_ida:

    data_jogo = jogo.find("div", class_="text-1 m-b-10 text-center uppercase").text.strip()
    link_jogo = jogo.find("a", class_="no-underline").get("href")
    local_jogo = jogo.find("div", class_="text-1 m-t-10 text-center uppercase").text.strip()
    timeA_sigla = jogo.find('b', class_='text-2 pull-right p-t-15 p-r-10 hidden-lg').text.strip()
    timeA_nome = jogo.find('div', class_='text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm hidden-md').text.strip()
    timeB_sigla = jogo.find('b', class_='text-2 pull-left p-t-15 p-l-10 hidden-lg').text.strip()
    timeB_nome = jogo.find('div', class_='text-2 pull-left p-t-15 p-l-20 hidden-xs hidden-sm hidden-md').text.strip()
    try:
        resultado = jogo.find('div', class_='text-3 p-t-15').text.strip()
    except:
        resultado = "Jogo ainda não ocorreu"
    if resultado == "x":
        resultado = "Jogo ainda não ocorreu"
    
    # Add the dictionary to the data list
    copabrasil_lista_timeA_timeB_resultado.append({
        'data_jogo': data_jogo,
        'link_jogo': link_jogo,
        'local_jogo': local_jogo,
        'timeA_sigla': timeA_sigla,
        'timeA_nome': timeA_nome,
        'timeB_sigla': timeB_sigla,
        'timeB_nome': timeB_nome,
        'resultado': resultado,
        'fase': "3ª Fase",
        'ida ou volta': "Ida",
        'torneio': "Copa do Brasil",
    })


copabrasil_lista_timeA_timeB_resultado

[{'data_jogo': 'Ter, 11/04/2023 16:30\r\n                - GRUPO 66 |  Jogo: 66',
  'link_jogo': 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/66?ref=linha',
  'local_jogo': 'Raulino de Oliveira - Volta Redonda - RJ\nComo foi o jogo',
  'timeA_sigla': 'VOL',
  'timeA_nome': 'Volta Redonda - RJ',
  'timeB_sigla': 'BAH',
  'timeB_nome': 'Bahia - BA',
  'resultado': '1 x 2',
  'fase': '3ª Fase',
  'ida ou volta': 'Ida',
  'torneio': 'Copa do Brasil'},
 {'data_jogo': 'Ter, 11/04/2023 19:00\r\n                - GRUPO 61 |  Jogo: 61',
  'link_jogo': 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/61?ref=linha',
  'local_jogo': 'Santa Cruz - Ribeirao Preto - SP\nComo foi o jogo',
  'timeA_sigla': 'BOT',
  'timeA_nome': 'Botafogo - SP',
  'timeB_sigla': 'SAN',
  'timeB_nome': 'Santos - SP',
  'resultado': '0 x 2',
  'fase': '3ª Fase',
  'ida ou volta': 'Ida',
  'torneio': 'Copa do Brasil'},
 {'data_jogo': 'Ter, 11/04/2023 20

In [771]:
# # Convert the data to a pandas DataFrame
copabrasil_base_lista_timeA_timeB_resultado_3fase = pd.DataFrame(copabrasil_lista_timeA_timeB_resultado)

copabrasil_base_lista_timeA_timeB_resultado_3fase

,data_jogo,link_jogo,local_jogo,timeA_sigla,timeA_nome,timeB_sigla,timeB_nome,resultado,fase,ida ou volta,torneio
0,"Ter, 11/04/2023 16:30\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Raulino de Oliveira - Volta Redonda - RJ\nComo...,VOL,Volta Redonda - RJ,BAH,Bahia - BA,1 x 2,3ª Fase,Ida,Copa do Brasil
1,"Ter, 11/04/2023 19:00\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Santa Cruz - Ribeirao Preto - SP\nComo foi o jogo,BOT,Botafogo - SP,SAN,Santos - SP,0 x 2,3ª Fase,Ida,Copa do Brasil
2,"Ter, 11/04/2023 20:00\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Beira-Rio - Porto Alegre - RS\nComo foi o jogo,INT,Internacional - RS,CSA,Csa - AL,2 x 1,3ª Fase,Ida,Copa do Brasil
3,"Ter, 11/04/2023 21:30\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Morumbi - Sao Paulo - SP\nComo foi o jogo,SAO,São Paulo - SP,ITU,Ituano - SP,0 x 0,3ª Fase,Ida,Copa do Brasil
4,"Ter, 11/04/2023 21:30\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Arena Castelão - Fortaleza - CE\nComo foi o jogo,FOR,Fortaleza - CE,AGU,Aguia de Maraba - PA,6 x 1,3ª Fase,Ida,Copa do Brasil
5,"Qua, 12/04/2023 16:30\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Raulino de Oliveira - Volta Redonda - RJ\nComo...,NOV,Nova Iguaçu - RJ,AME,América Fc Saf - MG,1 x 2,3ª Fase,Ida,Copa do Brasil
6,"Qua, 12/04/2023 19:00\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Couto Pereira - Curitiba - PR\nComo foi o jogo,COR,Coritiba - PR,SPO,Sport - PE,3 x 3,3ª Fase,Ida,Copa do Brasil
7,"Qua, 12/04/2023 19:30\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Rei Pelé - Maceio - AL\nComo foi o jogo,CRB,Crb - AL,ATH,Athletico Paranaense - PR,1 x 0,3ª Fase,Ida,Copa do Brasil
8,"Qua, 12/04/2023 19:30\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Maracanã - Rio de Janeiro - RJ\nComo foi o jogo,FLU,Fluminense - RJ,PAY,Paysandu - PA,3 x 0,3ª Fase,Ida,Copa do Brasil
9,"Qua, 12/04/2023 20:00\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Allianz Parque - Sao Paulo - SP\nComo foi o jogo,PAL,Palmeiras - SP,TOM,Tombense - MG,4 x 2,3ª Fase,Ida,Copa do Brasil


#### Volta

In [772]:
copabrasil_grupos_3fase_volta = copabrasil_soup_3fase.find_all('div', class_='col-md-6 p-b-15 p-t-20 block')

# len(copabrasil_grupos_oitavas_volta)
copabrasil_grupos_3fase_volta[0]

<div class="col-md-6 p-b-15 p-t-20 block">
<div class="text-1 m-b-10 text-center uppercase">
                      Qui, 27/04/2023 19:00
                    - GRUPO 66 |  Jogo: 82 
                  </div>
<a class="no-underline" href="https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/82?ref=linha" rel="noopener" target="_blank">
<div class="clearfix">
<div class="col-xs-5 p-t-10 nopadding">
<img alt="Bahia - BA" class="icon escudo x54 pull-right" onerror="this.src='https://conteudo.cbf.com.br/cdn/imagens/escudos/empty.jpg'" src="https://conteudo.cbf.com.br/cdn/imagens/escudos/61377ba.jpg?v=2023062419" title="Bahia - BA"/>
<b class="text-2 pull-right p-t-15 p-r-10 hidden-lg">BAH</b>
<div class="text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm hidden-md">Bahia - BA</div>
</div>
<div class="col-xs-2 color-blue text-center nopadding">
<div class="text-3 p-t-15">
<b>4 x 0</b>
</div>
</div>
<div class="col-xs-5 p-t-10 nopadding">
<img alt="Volta Redonda - R

In [773]:
# Initialize a list to store the extracted data
copabrasil_lista_timeA_timeB_resultado = []

# Iterate over the div elements
for jogo in copabrasil_grupos_3fase_volta:

    data_jogo = jogo.find("div", class_="text-1 m-b-10 text-center uppercase").text.strip()
    link_jogo = jogo.find("a", class_="no-underline").get("href")
    local_jogo = jogo.find("div", class_="text-1 m-t-10 text-center uppercase").text.strip()
    timeA_sigla = jogo.find('b', class_='text-2 pull-right p-t-15 p-r-10 hidden-lg').text.strip()
    timeA_nome = jogo.find('div', class_='text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm hidden-md').text.strip()
    timeB_sigla = jogo.find('b', class_='text-2 pull-left p-t-15 p-l-10 hidden-lg').text.strip()
    timeB_nome = jogo.find('div', class_='text-2 pull-left p-t-15 p-l-20 hidden-xs hidden-sm hidden-md').text.strip()
    try:
        resultado = jogo.find('div', class_='text-3 p-t-15').text.strip()
    except:
        resultado = "Jogo ainda não ocorreu"
    if resultado == "x":
        resultado = "Jogo ainda não ocorreu"
    
    # Add the dictionary to the data list
    copabrasil_lista_timeA_timeB_resultado.append({
        'data_jogo': data_jogo,
        'link_jogo': link_jogo,
        'local_jogo': local_jogo,
        'timeA_sigla': timeA_sigla,
        'timeA_nome': timeA_nome,
        'timeB_sigla': timeB_sigla,
        'timeB_nome': timeB_nome,
        'resultado': resultado,
        'fase': "3ª Fase",
        'ida ou volta': "Volta",
        'torneio': "Copa do Brasil",
    })


copabrasil_lista_timeA_timeB_resultado

[{'data_jogo': 'Qui, 27/04/2023 19:00\r\n                    - GRUPO 66 |  Jogo: 82',
  'link_jogo': 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/82?ref=linha',
  'local_jogo': 'Arena Fonte Nova - Salvador - BA\nComo foi o jogo',
  'timeA_sigla': 'BAH',
  'timeA_nome': 'Bahia - BA',
  'timeB_sigla': 'VOL',
  'timeB_nome': 'Volta Redonda - RJ',
  'resultado': '4 x 0',
  'fase': '3ª Fase',
  'ida ou volta': 'Volta',
  'torneio': 'Copa do Brasil'},
 {'data_jogo': 'Qua, 26/04/2023 19:00\r\n                    - GRUPO 61 |  Jogo: 77',
  'link_jogo': 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/77?ref=linha',
  'local_jogo': 'Vila Belmiro - Santos - SP\nComo foi o jogo',
  'timeA_sigla': 'SAN',
  'timeA_nome': 'Santos - SP',
  'timeB_sigla': 'BOT',
  'timeB_nome': 'Botafogo - SP',
  'resultado': '1 x 0',
  'fase': '3ª Fase',
  'ida ou volta': 'Volta',
  'torneio': 'Copa do Brasil'},
 {'data_jogo': 'Qui, 27/04/2023 20:0

In [774]:
# # Convert the data to a pandas DataFrame
copabrasil_base_lista_timeA_timeB_resultado_3fase = pd.DataFrame(copabrasil_lista_timeA_timeB_resultado)

copabrasil_base_lista_timeA_timeB_resultado_3fase

,data_jogo,link_jogo,local_jogo,timeA_sigla,timeA_nome,timeB_sigla,timeB_nome,resultado,fase,ida ou volta,torneio
0,"Qui, 27/04/2023 19:00\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Arena Fonte Nova - Salvador - BA\nComo foi o jogo,BAH,Bahia - BA,VOL,Volta Redonda - RJ,4 x 0,3ª Fase,Volta,Copa do Brasil
1,"Qua, 26/04/2023 19:00\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Vila Belmiro - Santos - SP\nComo foi o jogo,SAN,Santos - SP,BOT,Botafogo - SP,1 x 0,3ª Fase,Volta,Copa do Brasil
2,"Qui, 27/04/2023 20:00\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Rei Pelé - Maceio - AL\nComo foi o jogo,CSA,Csa - AL,INT,Internacional - RS,(6)2 x 1(7),3ª Fase,Volta,Copa do Brasil
3,"Ter, 25/04/2023 21:30\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Novelli Junior - Itu - SP\nComo foi o jogo,ITU,Ituano - SP,SAO,São Paulo - SP,0 x 1,3ª Fase,Volta,Copa do Brasil
4,"Qua, 26/04/2023 19:30\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Mangueirão - Belem - PA\nComo foi o jogo,AGU,Aguia de Maraba - PA,FOR,Fortaleza - CE,0 x 2,3ª Fase,Volta,Copa do Brasil
5,"Qua, 26/04/2023 21:30\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Independência - Belo Horizonte - MG\nComo foi ...,AME,América Fc Saf - MG,NOV,Nova Iguaçu - RJ,5 x 0,3ª Fase,Volta,Copa do Brasil
6,"Qua, 26/04/2023 19:00\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Ilha do Retiro - Recife - PE\nComo foi o jogo,SPO,Sport - PE,COR,Coritiba - PR,2 x 0,3ª Fase,Volta,Copa do Brasil
7,"Ter, 25/04/2023 21:30\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Arena da Baixada - Curitiba - PR\nComo foi o jogo,ATH,Athletico Paranaense - PR,CRB,Crb - AL,(4)2 x 1(2),3ª Fase,Volta,Copa do Brasil
8,"Ter, 25/04/2023 20:00\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Mangueirão - Belem - PA\nComo foi o jogo,PAY,Paysandu - PA,FLU,Fluminense - RJ,0 x 3,3ª Fase,Volta,Copa do Brasil
9,"Qua, 26/04/2023 20:00\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Parque do Sabiá - Uberlandia - MG\nComo foi o ...,TOM,Tombense - MG,PAL,Palmeiras - SP,1 x 1,3ª Fase,Volta,Copa do Brasil


### Oitavas de final

In [775]:
# Make a request to the webpage
copabrasil_url_oitavas = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023?phase=1696'
copabrasil_response_oitavas = requests.get(copabrasil_url_oitavas)

# Create a BeautifulSoup object with the HTML content
copabrasil_soup_oitavas = BeautifulSoup(copabrasil_response_oitavas.content, 'html.parser')

# soup

#### Ida

In [776]:
copabrasil_grupos_oitavas_ida = copabrasil_soup_oitavas.find_all('div', class_='col-md-6 p-b-15 p-t-20 block border-right')

# len(copabrasil_grupos_oitavas_ida)
copabrasil_grupos_oitavas_ida[0]

<div class="col-md-6 p-b-15 p-t-20 block border-right">
<div class="text-1 m-b-10 text-center uppercase">
                  Ter, 16/05/2023 21:00
                - GRUPO 80 |  Jogo: 96 
              </div>
<a class="no-underline" href="https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/96?ref=linha" rel="noopener" target="_blank">
<div class="clearfix">
<div class="col-xs-5 p-t-10 nopadding">
<img alt="Fluminense - RJ" class="icon escudo x54 pull-right" onerror="this.src='https://conteudo.cbf.com.br/cdn/imagens/escudos/empty.jpg'" src="https://conteudo.cbf.com.br/cdn/imagens/escudos/00009rj.jpg?v=2023062419" title="Fluminense - RJ"/>
<b class="text-2 pull-right p-t-15 p-r-10 hidden-lg">FLU</b>
<div class="text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm hidden-md">Fluminense - RJ</div>
</div>
<div class="col-xs-2 color-blue text-center nopadding">
<div class="text-3 p-t-15">
<b>0 x 0</b>
</div>
</div>
<div class="col-xs-5 p-t-10 nopadding">
<img alt="F

In [777]:
# Initialize a list to store the extracted data
copabrasil_lista_timeA_timeB_resultado = []

# Iterate over the div elements
for jogo in copabrasil_grupos_oitavas_ida:

    data_jogo = jogo.find("div", class_="text-1 m-b-10 text-center uppercase").text.strip()
    link_jogo = jogo.find("a", class_="no-underline").get("href")
    local_jogo = jogo.find("div", class_="text-1 m-t-10 text-center uppercase").text.strip()
    timeA_sigla = jogo.find('b', class_='text-2 pull-right p-t-15 p-r-10 hidden-lg').text.strip()
    timeA_nome = jogo.find('div', class_='text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm hidden-md').text.strip()
    timeB_sigla = jogo.find('b', class_='text-2 pull-left p-t-15 p-l-10 hidden-lg').text.strip()
    timeB_nome = jogo.find('div', class_='text-2 pull-left p-t-15 p-l-20 hidden-xs hidden-sm hidden-md').text.strip()
    try:
        resultado = jogo.find('div', class_='text-3 p-t-15').text.strip()
    except:
        resultado = "Jogo ainda não ocorreu"
    if resultado == "x":
        resultado = "Jogo ainda não ocorreu"
    
    # Add the dictionary to the data list
    copabrasil_lista_timeA_timeB_resultado.append({
        'data_jogo': data_jogo,
        'link_jogo': link_jogo,
        'local_jogo': local_jogo,
        'timeA_sigla': timeA_sigla,
        'timeA_nome': timeA_nome,
        'timeB_sigla': timeB_sigla,
        'timeB_nome': timeB_nome,
        'resultado': resultado,
        'fase': "Oitavas de Final",
        'ida ou volta': "Ida",
        'torneio': "Copa do Brasil",
    })


copabrasil_lista_timeA_timeB_resultado

[{'data_jogo': 'Ter, 16/05/2023 21:00\r\n                - GRUPO 80 |  Jogo: 96',
  'link_jogo': 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/96?ref=linha',
  'local_jogo': 'Maracanã - Rio de Janeiro - RJ\nComo foi o jogo',
  'timeA_sigla': 'FLU',
  'timeA_nome': 'Fluminense - RJ',
  'timeB_sigla': 'FLA',
  'timeB_nome': 'Flamengo - RJ',
  'resultado': '0 x 0',
  'fase': 'Oitavas de Final',
  'ida ou volta': 'Ida',
  'torneio': 'Copa do Brasil'},
 {'data_jogo': 'Qua, 17/05/2023 19:00\r\n                - GRUPO 81 |  Jogo: 97',
  'link_jogo': 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/97?ref=linha',
  'local_jogo': 'Vila Belmiro - Santos - SP\nComo foi o jogo',
  'timeA_sigla': 'SAN',
  'timeA_nome': 'Santos - SP',
  'timeB_sigla': 'BAH',
  'timeB_nome': 'Bahia - BA',
  'resultado': '0 x 0',
  'fase': 'Oitavas de Final',
  'ida ou volta': 'Ida',
  'torneio': 'Copa do Brasil'},
 {'data_jogo': 'Qua, 17/05/2023 19:

In [778]:
# # Convert the data to a pandas DataFrame
copabrasil_base_lista_timeA_timeB_resultado_oitavas_ida = pd.DataFrame(copabrasil_lista_timeA_timeB_resultado)

copabrasil_base_lista_timeA_timeB_resultado_oitavas_ida

,data_jogo,link_jogo,local_jogo,timeA_sigla,timeA_nome,timeB_sigla,timeB_nome,resultado,fase,ida ou volta,torneio
0,"Ter, 16/05/2023 21:00\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Maracanã - Rio de Janeiro - RJ\nComo foi o jogo,FLU,Fluminense - RJ,FLA,Flamengo - RJ,0 x 0,Oitavas de Final,Ida,Copa do Brasil
1,"Qua, 17/05/2023 19:00\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Vila Belmiro - Santos - SP\nComo foi o jogo,SAN,Santos - SP,BAH,Bahia - BA,0 x 0,Oitavas de Final,Ida,Copa do Brasil
2,"Qua, 17/05/2023 19:00\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Allianz Parque - Sao Paulo - SP\nComo foi o jogo,PAL,Palmeiras - SP,FOR,Fortaleza - CE,3 x 0,Oitavas de Final,Ida,Copa do Brasil
3,"Qua, 17/05/2023 19:30\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Arena do Grêmio - Porto Alegre - RS\nComo foi ...,GRE,Grêmio - RS,CRU,Cruzeiro Saf - MG,1 x 1,Oitavas de Final,Ida,Copa do Brasil
4,"Qua, 17/05/2023 20:00\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Ilha do Retiro - Recife - PE\nComo foi o jogo,SPO,Sport - PE,SAO,São Paulo - SP,0 x 2,Oitavas de Final,Ida,Copa do Brasil
5,"Qua, 17/05/2023 21:30\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Independência - Belo Horizonte - MG\nComo foi ...,AME,América Fc Saf - MG,INT,Internacional - RS,2 x 0,Oitavas de Final,Ida,Copa do Brasil
6,"Qua, 17/05/2023 21:30\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Arena da Baixada - Curitiba - PR\nComo foi o jogo,ATH,Athletico Paranaense - PR,BOT,Botafogo - RJ,3 x 2,Oitavas de Final,Ida,Copa do Brasil
7,"Qua, 17/05/2023 21:30\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Mineirão - Belo Horizonte - MG\nComo foi o jogo,ATL,Atlético Mineiro - MG,COR,Corinthians - SP,2 x 0,Oitavas de Final,Ida,Copa do Brasil


#### Volta

In [779]:
copabrasil_grupos_oitavas_volta = copabrasil_soup_oitavas.find_all('div', class_='col-md-6 p-b-15 p-t-20 block')

# len(copabrasil_grupos_oitavas_volta)
copabrasil_grupos_oitavas_volta[0]

<div class="col-md-6 p-b-15 p-t-20 block">
<div class="text-1 m-b-10 text-center uppercase">
                      Qui, 01/06/2023 20:00
                    - GRUPO 80 |  Jogo: 104 
                  </div>
<a class="no-underline" href="https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/104?ref=linha" rel="noopener" target="_blank">
<div class="clearfix">
<div class="col-xs-5 p-t-10 nopadding">
<img alt="Flamengo - RJ" class="icon escudo x54 pull-right" onerror="this.src='https://conteudo.cbf.com.br/cdn/imagens/escudos/empty.jpg'" src="https://conteudo.cbf.com.br/cdn/imagens/escudos/00006rj.jpg?v=2023062419" title="Flamengo - RJ"/>
<b class="text-2 pull-right p-t-15 p-r-10 hidden-lg">FLA</b>
<div class="text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm hidden-md">Flamengo - RJ</div>
</div>
<div class="col-xs-2 color-blue text-center nopadding">
<div class="text-3 p-t-15">
<b>2 x 0</b>
</div>
</div>
<div class="col-xs-5 p-t-10 nopadding">
<img alt="Flumin

In [780]:
# Initialize a list to store the extracted data
copabrasil_lista_timeA_timeB_resultado = []

# Iterate over the div elements
for jogo in copabrasil_grupos_oitavas_volta:

    data_jogo = jogo.find("div", class_="text-1 m-b-10 text-center uppercase").text.strip()
    link_jogo = jogo.find("a", class_="no-underline").get("href")
    local_jogo = jogo.find("div", class_="text-1 m-t-10 text-center uppercase").text.strip()
    timeA_sigla = jogo.find('b', class_='text-2 pull-right p-t-15 p-r-10 hidden-lg').text.strip()
    timeA_nome = jogo.find('div', class_='text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm hidden-md').text.strip()
    timeB_sigla = jogo.find('b', class_='text-2 pull-left p-t-15 p-l-10 hidden-lg').text.strip()
    timeB_nome = jogo.find('div', class_='text-2 pull-left p-t-15 p-l-20 hidden-xs hidden-sm hidden-md').text.strip()
    try:
        resultado = jogo.find('div', class_='text-3 p-t-15').text.strip()
    except:
        resultado = "Jogo ainda não ocorreu"
    if resultado == "x":
        resultado = "Jogo ainda não ocorreu"
    
    # Add the dictionary to the data list
    copabrasil_lista_timeA_timeB_resultado.append({
        'data_jogo': data_jogo,
        'link_jogo': link_jogo,
        'local_jogo': local_jogo,
        'timeA_sigla': timeA_sigla,
        'timeA_nome': timeA_nome,
        'timeB_sigla': timeB_sigla,
        'timeB_nome': timeB_nome,
        'resultado': resultado,
        'fase': "Oitavas de Final",
        'ida ou volta': "Volta",
        'torneio': "Copa do Brasil",
    })


copabrasil_lista_timeA_timeB_resultado

[{'data_jogo': 'Qui, 01/06/2023 20:00\r\n                    - GRUPO 80 |  Jogo: 104',
  'link_jogo': 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/104?ref=linha',
  'local_jogo': 'Maracanã - Rio de Janeiro - RJ\nComo foi o jogo',
  'timeA_sigla': 'FLA',
  'timeA_nome': 'Flamengo - RJ',
  'timeB_sigla': 'FLU',
  'timeB_nome': 'Fluminense - RJ',
  'resultado': '2 x 0',
  'fase': 'Oitavas de Final',
  'ida ou volta': 'Volta',
  'torneio': 'Copa do Brasil'},
 {'data_jogo': 'Qua, 31/05/2023 19:00\r\n                    - GRUPO 81 |  Jogo: 105',
  'link_jogo': 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/105?ref=linha',
  'local_jogo': 'Arena Fonte Nova - Salvador - BA\nComo foi o jogo',
  'timeA_sigla': 'BAH',
  'timeA_nome': 'Bahia - BA',
  'timeB_sigla': 'SAN',
  'timeB_nome': 'Santos - SP',
  'resultado': '(4)1 x 1(3)',
  'fase': 'Oitavas de Final',
  'ida ou volta': 'Volta',
  'torneio': 'Copa do Brasil'},
 {'data

In [781]:
# # Convert the data to a pandas DataFrame
copabrasil_base_lista_timeA_timeB_resultado_oitavas_volta = pd.DataFrame(copabrasil_lista_timeA_timeB_resultado)

copabrasil_base_lista_timeA_timeB_resultado_oitavas_volta

,data_jogo,link_jogo,local_jogo,timeA_sigla,timeA_nome,timeB_sigla,timeB_nome,resultado,fase,ida ou volta,torneio
0,"Qui, 01/06/2023 20:00\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Maracanã - Rio de Janeiro - RJ\nComo foi o jogo,FLA,Flamengo - RJ,FLU,Fluminense - RJ,2 x 0,Oitavas de Final,Volta,Copa do Brasil
1,"Qua, 31/05/2023 19:00\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Arena Fonte Nova - Salvador - BA\nComo foi o jogo,BAH,Bahia - BA,SAN,Santos - SP,(4)1 x 1(3),Oitavas de Final,Volta,Copa do Brasil
2,"Qua, 31/05/2023 19:00\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Arena Castelão - Fortaleza - CE\nComo foi o jogo,FOR,Fortaleza - CE,PAL,Palmeiras - SP,1 x 0,Oitavas de Final,Volta,Copa do Brasil
3,"Qua, 31/05/2023 20:00\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Mineirão - Belo Horizonte - MG\nComo foi o jogo,CRU,Cruzeiro Saf - MG,GRE,Grêmio - RS,0 x 1,Oitavas de Final,Volta,Copa do Brasil
4,"Qui, 01/06/2023 19:30\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Morumbi - Sao Paulo - SP\nComo foi o jogo,SAO,São Paulo - SP,SPO,Sport - PE,(5)1 x 3(3),Oitavas de Final,Volta,Copa do Brasil
5,"Qua, 31/05/2023 21:30\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Beira-Rio - Porto Alegre - RS\nComo foi o jogo,INT,Internacional - RS,AME,América Fc Saf - MG,(4)3 x 1(5),Oitavas de Final,Volta,Copa do Brasil
6,"Qua, 31/05/2023 21:30\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Nilton Santos - Rio de Janeiro - RJ\nComo foi ...,BOT,Botafogo - RJ,ATH,Athletico Paranaense - PR,(2)1 x 0(4),Oitavas de Final,Volta,Copa do Brasil
7,"Qua, 31/05/2023 21:30\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Neo Química Arena - Sao Paulo - SP\nComo foi o...,COR,Corinthians - SP,ATL,Atlético Mineiro - MG,(3)2 x 0(1),Oitavas de Final,Volta,Copa do Brasil


### Quartas de final

In [782]:
# Make a request to the webpage
copabrasil_url_quartas = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023?phase=1703'
copabrasil_response_quartas = requests.get(copabrasil_url_quartas)

# Create a BeautifulSoup object with the HTML content
copabrasil_soup_quartas = BeautifulSoup(copabrasil_response_quartas.content, 'html.parser')

# soup

#### Ida

In [783]:
copabrasil_grupos_quartas_ida = copabrasil_soup_quartas.find_all('div', class_='col-md-6 p-b-15 p-t-20 block border-right')

# len(copabrasil_grupos_oitavas_ida)
copabrasil_grupos_quartas_ida[0]

<div class="col-md-6 p-b-15 p-t-20 block border-right">
<div class="text-1 m-b-10 text-center uppercase">
                  Ter, 04/07/2023 21:00
                - GRUPO 87 |  Jogo: 111 
              </div>
<a class="no-underline" href="https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/111?ref=linha" rel="noopener" target="_blank">
<div class="clearfix">
<div class="col-xs-5 p-t-10 nopadding">
<img alt="Bahia - BA" class="icon escudo x54 pull-right" onerror="this.src='https://conteudo.cbf.com.br/cdn/imagens/escudos/empty.jpg'" src="https://conteudo.cbf.com.br/cdn/imagens/escudos/61377ba.jpg?v=2023062419" title="Bahia - BA"/>
<b class="text-2 pull-right p-t-15 p-r-10 hidden-lg">BAH</b>
<div class="text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm hidden-md">Bahia - BA</div>
</div>
<div class="col-xs-2 color-blue text-center nopadding">
<div class="text-3 p-t-15">
<b>x</b>
</div>
</div>
<div class="col-xs-5 p-t-10 nopadding">
<img alt="Grêmio - RS" class

In [784]:
# Initialize a list to store the extracted data
copabrasil_lista_timeA_timeB_resultado = []

# Iterate over the div elements
for jogo in copabrasil_grupos_quartas_ida:

    data_jogo = jogo.find("div", class_="text-1 m-b-10 text-center uppercase").text.strip()
    link_jogo = jogo.find("a", class_="no-underline").get("href")
    local_jogo = jogo.find("div", class_="text-1 m-t-10 text-center uppercase").text.strip()
    timeA_sigla = jogo.find('b', class_='text-2 pull-right p-t-15 p-r-10 hidden-lg').text.strip()
    timeA_nome = jogo.find('div', class_='text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm hidden-md').text.strip()
    timeB_sigla = jogo.find('b', class_='text-2 pull-left p-t-15 p-l-10 hidden-lg').text.strip()
    timeB_nome = jogo.find('div', class_='text-2 pull-left p-t-15 p-l-20 hidden-xs hidden-sm hidden-md').text.strip()
    try:
        resultado = jogo.find('div', class_='text-3 p-t-15').text.strip()
    except:
        resultado = "Jogo ainda não ocorreu"
    if resultado == "x":
        resultado = "Jogo ainda não ocorreu"

    
    # Add the dictionary to the data list
    copabrasil_lista_timeA_timeB_resultado.append({
        'data_jogo': data_jogo,
        'link_jogo': link_jogo,
        'local_jogo': local_jogo,
        'timeA_sigla': timeA_sigla,
        'timeA_nome': timeA_nome,
        'timeB_sigla': timeB_sigla,
        'timeB_nome': timeB_nome,
        'resultado': resultado,
        'fase': "Quartas de Final",
        'ida ou volta': "Ida",
        'torneio': "Copa do Brasil",
    })


copabrasil_lista_timeA_timeB_resultado

[{'data_jogo': 'Ter, 04/07/2023 21:00\r\n                - GRUPO 87 |  Jogo: 111',
  'link_jogo': 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/111?ref=linha',
  'local_jogo': 'Arena Fonte Nova - Salvador - BA\nDetalhes do jogo',
  'timeA_sigla': 'BAH',
  'timeA_nome': 'Bahia - BA',
  'timeB_sigla': 'GRE',
  'timeB_nome': 'Grêmio - RS',
  'resultado': 'Jogo ainda não ocorreu',
  'fase': 'Quartas de Final',
  'ida ou volta': 'Ida',
  'torneio': 'Copa do Brasil'},
 {'data_jogo': 'Qua, 05/07/2023 19:30\r\n                - GRUPO 86 |  Jogo: 110',
  'link_jogo': 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/110?ref=linha',
  'local_jogo': 'Morumbi - Sao Paulo - SP\nDetalhes do jogo',
  'timeA_sigla': 'SAO',
  'timeA_nome': 'São Paulo - SP',
  'timeB_sigla': 'PAL',
  'timeB_nome': 'Palmeiras - SP',
  'resultado': 'Jogo ainda não ocorreu',
  'fase': 'Quartas de Final',
  'ida ou volta': 'Ida',
  'torneio': 'Copa do Brasi

In [785]:
# # Convert the data to a pandas DataFrame
copabrasil_base_lista_timeA_timeB_resultado_quartas_ida = pd.DataFrame(copabrasil_lista_timeA_timeB_resultado)

copabrasil_base_lista_timeA_timeB_resultado_quartas_ida

,data_jogo,link_jogo,local_jogo,timeA_sigla,timeA_nome,timeB_sigla,timeB_nome,resultado,fase,ida ou volta,torneio
0,"Ter, 04/07/2023 21:00\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Arena Fonte Nova - Salvador - BA\nDetalhes do ...,BAH,Bahia - BA,GRE,Grêmio - RS,Jogo ainda não ocorreu,Quartas de Final,Ida,Copa do Brasil
1,"Qua, 05/07/2023 19:30\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Morumbi - Sao Paulo - SP\nDetalhes do jogo,SAO,São Paulo - SP,PAL,Palmeiras - SP,Jogo ainda não ocorreu,Quartas de Final,Ida,Copa do Brasil
2,"Qua, 05/07/2023 21:30\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Independência - Belo Horizonte - MG\nDetalhes ...,AME,América Fc Saf - MG,COR,Corinthians - SP,Jogo ainda não ocorreu,Quartas de Final,Ida,Copa do Brasil
3,"Qua, 05/07/2023 21:30\r\n - GRU...",https://www.cbf.com.br/futebol-brasileiro/comp...,Maracanã - Rio de Janeiro - RJ\nDetalhes do jogo,FLA,Flamengo - RJ,ATH,Athletico Paranaense - PR,Jogo ainda não ocorreu,Quartas de Final,Ida,Copa do Brasil


#### Volta

In [786]:
copabrasil_grupos_quartas_volta = copabrasil_soup_quartas.find_all('div', class_='col-md-6 p-b-15 p-t-20 block')

# len(copabrasil_grupos_oitavas_volta)
copabrasil_grupos_quartas_volta[0]

<div class="col-md-6 p-b-15 p-t-20 block">
<div class="text-1 m-b-10 text-center uppercase">
                      Qua, 12/07/2023 19:00
                    - GRUPO 87 |  Jogo: 115 
                  </div>
<a class="no-underline" href="https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/115?ref=linha" rel="noopener" target="_blank">
<div class="clearfix">
<div class="col-xs-5 p-t-10 nopadding">
<img alt="Grêmio - RS" class="icon escudo x54 pull-right" onerror="this.src='https://conteudo.cbf.com.br/cdn/imagens/escudos/empty.jpg'" src="https://conteudo.cbf.com.br/cdn/imagens/escudos/00008rs.jpg?v=2023062419" title="Grêmio - RS"/>
<b class="text-2 pull-right p-t-15 p-r-10 hidden-lg">GRE</b>
<div class="text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm hidden-md">Grêmio - RS</div>
</div>
<div class="col-xs-2 color-blue text-center nopadding">
<div class="text-3 p-t-15">
<b>x</b>
</div>
</div>
<div class="col-xs-5 p-t-10 nopadding">
<img alt="Bahia - BA" clas

In [787]:
# Initialize a list to store the extracted data
copabrasil_lista_timeA_timeB_resultado = []

# Iterate over the div elements
for jogo in copabrasil_grupos_quartas_volta:

    data_jogo = jogo.find("div", class_="text-1 m-b-10 text-center uppercase").text.strip()
    link_jogo = jogo.find("a", class_="no-underline").get("href")
    local_jogo = jogo.find("div", class_="text-1 m-t-10 text-center uppercase").text.strip()
    timeA_sigla = jogo.find('b', class_='text-2 pull-right p-t-15 p-r-10 hidden-lg').text.strip()
    timeA_nome = jogo.find('div', class_='text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm hidden-md').text.strip()
    timeB_sigla = jogo.find('b', class_='text-2 pull-left p-t-15 p-l-10 hidden-lg').text.strip()
    timeB_nome = jogo.find('div', class_='text-2 pull-left p-t-15 p-l-20 hidden-xs hidden-sm hidden-md').text.strip()
    try:
        resultado = jogo.find('div', class_='text-3 p-t-15').text.strip()
    except:
        resultado = "Jogo ainda não ocorreu"
    if resultado == "x":
        resultado = "Jogo ainda não ocorreu"
    
    # Add the dictionary to the data list
    copabrasil_lista_timeA_timeB_resultado.append({
        'data_jogo': data_jogo,
        'link_jogo': link_jogo,
        'local_jogo': local_jogo,
        'timeA_sigla': timeA_sigla,
        'timeA_nome': timeA_nome,
        'timeB_sigla': timeB_sigla,
        'timeB_nome': timeB_nome,
        'resultado': resultado,
        'fase': "Quartas de Final",
        'ida ou volta': "Volta",
        'torneio': "Copa do Brasil",
    })


copabrasil_lista_timeA_timeB_resultado

[{'data_jogo': 'Qua, 12/07/2023 19:00\r\n                    - GRUPO 87 |  Jogo: 115',
  'link_jogo': 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/115?ref=linha',
  'local_jogo': 'Arena do Grêmio - Porto Alegre - RS\nDetalhes do jogo',
  'timeA_sigla': 'GRE',
  'timeA_nome': 'Grêmio - RS',
  'timeB_sigla': 'BAH',
  'timeB_nome': 'Bahia - BA',
  'resultado': 'Jogo ainda não ocorreu',
  'fase': 'Quartas de Final',
  'ida ou volta': 'Volta',
  'torneio': 'Copa do Brasil'},
 {'data_jogo': 'Qui, 13/07/2023 20:00\r\n                    - GRUPO 86 |  Jogo: 114',
  'link_jogo': 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023/114?ref=linha',
  'local_jogo': 'Allianz Parque - Sao Paulo - SP\nDetalhes do jogo',
  'timeA_sigla': 'PAL',
  'timeA_nome': 'Palmeiras - SP',
  'timeB_sigla': 'SAO',
  'timeB_nome': 'São Paulo - SP',
  'resultado': 'Jogo ainda não ocorreu',
  'fase': 'Quartas de Final',
  'ida ou volta': 'Volta',
  'to

In [788]:
# # Convert the data to a pandas DataFrame
copabrasil_base_lista_timeA_timeB_resultado_quartas_volta = pd.DataFrame(copabrasil_lista_timeA_timeB_resultado)

copabrasil_base_lista_timeA_timeB_resultado_quartas_volta

,data_jogo,link_jogo,local_jogo,timeA_sigla,timeA_nome,timeB_sigla,timeB_nome,resultado,fase,ida ou volta,torneio
0,"Qua, 12/07/2023 19:00\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Arena do Grêmio - Porto Alegre - RS\nDetalhes ...,GRE,Grêmio - RS,BAH,Bahia - BA,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil
1,"Qui, 13/07/2023 20:00\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Allianz Parque - Sao Paulo - SP\nDetalhes do jogo,PAL,Palmeiras - SP,SAO,São Paulo - SP,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil
2,"Qua, 12/07/2023 21:30\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Neo Química Arena - Sao Paulo - SP\nDetalhes d...,COR,Corinthians - SP,AME,América Fc Saf - MG,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil
3,"Qua, 12/07/2023 21:30\r\n -...",https://www.cbf.com.br/futebol-brasileiro/comp...,Arena da Baixada - Curitiba - PR\nDetalhes do ...,ATH,Athletico Paranaense - PR,FLA,Flamengo - RJ,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil


# Tratamento dos dados (site CBF)

## Empilhamento de bases

In [806]:
base_jogos = pd.concat([
    brasileirao_base_timeA_timeB_resultado_datas_numeracao,
    copabrasil_base_lista_timeA_timeB_resultado_1fase,
    copabrasil_base_lista_timeA_timeB_resultado_2fase,
    copabrasil_base_lista_timeA_timeB_resultado_3fase,
    copabrasil_base_lista_timeA_timeB_resultado_oitavas_ida,
    copabrasil_base_lista_timeA_timeB_resultado_oitavas_volta,
    copabrasil_base_lista_timeA_timeB_resultado_quartas_ida,
    copabrasil_base_lista_timeA_timeB_resultado_quartas_volta,
]).reset_index().drop("index", axis = 1)

base_jogos

,link_jogo,timeA_sigla,timeA_nome,timeB_sigla,timeB_nome,resultado,fase,ida ou volta,torneio,data_jogo,local_jogo
0,https://www.cbf.com.br/futebol-brasileiro/comp...,AME,América Fc Saf - MG,FLU,Fluminense - RJ,0 x 3,NaN,Única,Brasileirão,"Sáb, 15/04/2023 16:00\r\n ...",Independência - Belo Horizonte - MG\nComo foi ...
1,https://www.cbf.com.br/futebol-brasileiro/comp...,PAL,Palmeiras - SP,CUI,Cuiabá Saf - MT,2 x 1,NaN,Única,Brasileirão,"Sáb, 15/04/2023 16:00\r\n ...",Allianz Parque - Sao Paulo - SP\nComo foi o jogo
2,https://www.cbf.com.br/futebol-brasileiro/comp...,RED,Red Bull Bragantino - SP,BAH,Bahia - BA,2 x 1,NaN,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",Nabi Abi Chedid - Braganca Paulista - SP\nComo...
3,https://www.cbf.com.br/futebol-brasileiro/comp...,ATH,Athletico Paranaense - PR,GOI,Goiás - GO,2 x 0,NaN,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",Ligga Arena - Curitiba - PR\nComo foi o jogo
4,https://www.cbf.com.br/futebol-brasileiro/comp...,FOR,Fortaleza - CE,INT,Internacional - RS,1 x 1,NaN,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",Arena Castelão - Fortaleza - CE\nComo foi o jogo
...,...,...,...,...,...,...,...,...,...,...,...
475,https://www.cbf.com.br/futebol-brasileiro/comp...,FLA,Flamengo - RJ,ATH,Athletico Paranaense - PR,Jogo ainda não ocorreu,Quartas de Final,Ida,Copa do Brasil,"Qua, 05/07/2023 21:30\r\n - GRU...",Maracanã - Rio de Janeiro - RJ\nDetalhes do jogo
476,https://www.cbf.com.br/futebol-brasileiro/comp...,GRE,Grêmio - RS,BAH,Bahia - BA,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qua, 12/07/2023 19:00\r\n -...",Arena do Grêmio - Porto Alegre - RS\nDetalhes ...
477,https://www.cbf.com.br/futebol-brasileiro/comp...,PAL,Palmeiras - SP,SAO,São Paulo - SP,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qui, 13/07/2023 20:00\r\n -...",Allianz Parque - Sao Paulo - SP\nDetalhes do jogo
478,https://www.cbf.com.br/futebol-brasileiro/comp...,COR,Corinthians - SP,AME,América Fc Saf - MG,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qua, 12/07/2023 21:30\r\n -...",Neo Química Arena - Sao Paulo - SP\nDetalhes d...


In [807]:
base_jogos_backup = base_jogos.copy()

## Ajustar nomes colunas

In [808]:
base_jogos.columns = [
    'Link para o jogo',
    'Time mandante (sigla)',
    'Time mandante (nome)',
    'Time visitante (sigla)',
    'Time visitante (nome)',
    'Resultado',
    'Fase do torneio',
    'Ida ou volta',
    'Torneio',
    'Data',
    'Local'
]

base_jogos.head()

,Link para o jogo,Time mandante (sigla),Time mandante (nome),Time visitante (sigla),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data,Local
0,https://www.cbf.com.br/futebol-brasileiro/comp...,AME,América Fc Saf - MG,FLU,Fluminense - RJ,0 x 3,NaN,Única,Brasileirão,"Sáb, 15/04/2023 16:00\r\n ...",Independência - Belo Horizonte - MG\nComo foi ...
1,https://www.cbf.com.br/futebol-brasileiro/comp...,PAL,Palmeiras - SP,CUI,Cuiabá Saf - MT,2 x 1,NaN,Única,Brasileirão,"Sáb, 15/04/2023 16:00\r\n ...",Allianz Parque - Sao Paulo - SP\nComo foi o jogo
2,https://www.cbf.com.br/futebol-brasileiro/comp...,RED,Red Bull Bragantino - SP,BAH,Bahia - BA,2 x 1,NaN,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",Nabi Abi Chedid - Braganca Paulista - SP\nComo...
3,https://www.cbf.com.br/futebol-brasileiro/comp...,ATH,Athletico Paranaense - PR,GOI,Goiás - GO,2 x 0,NaN,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",Ligga Arena - Curitiba - PR\nComo foi o jogo
4,https://www.cbf.com.br/futebol-brasileiro/comp...,FOR,Fortaleza - CE,INT,Internacional - RS,1 x 1,NaN,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",Arena Castelão - Fortaleza - CE\nComo foi o jogo


## Tratar coluna de data e hora

In [809]:
base_jogos["Data"].str[4:21].str.strip()[85:100]

85    04/06/2023 16:00
86    04/06/2023 16:00
87    04/06/2023 18:30
88    04/06/2023 18:30
89    05/06/2023 20:00
90    10/06/2023 16:00
91    10/06/2023 18:30
92    10/06/2023 18:30
93    10/06/2023 18:30
94    10/06/2023 21:00
95    11/06/2023 11:00
96    11/06/2023 16:00
97    11/06/2023 16:00
98    11/06/2023 18:30
99    11/06/2023 18:30
Name: Data, dtype: object

In [810]:
base_jogos["Data e Hora"] = pd.to_datetime(base_jogos["Data"].str[4:21].str.strip(), format = "%d/%m/%Y %H:%M", errors='coerce')
base_jogos["Ano"] = base_jogos["Data e Hora"].dt.year
base_jogos["Mês"] = base_jogos["Data e Hora"].dt.month
base_jogos["Dia"] = base_jogos["Data e Hora"].dt.day
base_jogos["Hora"] = base_jogos["Data e Hora"].dt.hour
base_jogos["Minuto"] = base_jogos["Data e Hora"].dt.minute

# pd.DataFrame(base_jogos.groupby(["Torneio", "Mês"]).count().iloc[:, 0])
# pd.DataFrame(base_jogos.groupby(["Torneio", "Hora"]).count().iloc[:, 0])
# pd.DataFrame(base_jogos.groupby(["Torneio", "Minuto"]).count().iloc[:, 0])
# base_jogos[base_jogos["Data e Hora"].isna()]
base_jogos[(~base_jogos["Data e Hora"].isna()) * (base_jogos["Torneio"] == "Copa do Brasil")]

,Link para o jogo,Time mandante (sigla),Time mandante (nome),Time visitante (sigla),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data,Local,Data e Hora,Ano,Mês,Dia,Hora,Minuto
380,https://www.cbf.com.br/futebol-brasileiro/comp...,MAR,Marcílio Dias - SC,CHA,Chapecoense - SC,1 x 0,1ª Fase,Única,Copa do Brasil,"Ter, 21/02/2023 19:00\r\n - GRUPO 1...",Hercílio Luz - Itajai - SC\nComo foi o jogo,2023-02-21 19:00:00,2023.0,2.0,21.0,19.0,0.0
381,https://www.cbf.com.br/futebol-brasileiro/comp...,FAL,Falcon Futebol Clube - SE,VOL,Volta Redonda - RJ,1 x 3,1ª Fase,Única,Copa do Brasil,"Qua, 22/02/2023 16:00\r\n - GRUPO 2...",Batistão - Aracaju - SE\nComo foi o jogo,2023-02-22 16:00:00,2023.0,2.0,22.0,16.0,0.0
382,https://www.cbf.com.br/futebol-brasileiro/comp...,MUT,Mutum Esporte Clube - MT,LON,Londrina - PR,4 x 2,1ª Fase,Única,Copa do Brasil,"Qua, 22/02/2023 17:00\r\n - GRUPO 4...",Valdir Doilho Wons - Nova Mutum - MT\nComo foi...,2023-02-22 17:00:00,2023.0,2.0,22.0,17.0,0.0
383,https://www.cbf.com.br/futebol-brasileiro/comp...,SAO,São Raimundo - RR,CUI,Cuiabá Saf - MT,4 x 3,1ª Fase,Única,Copa do Brasil,"Qua, 22/02/2023 19:00\r\n - GRUPO 3...",Canarinho - Boa Vista - RR\nComo foi o jogo,2023-02-22 19:00:00,2023.0,2.0,22.0,19.0,0.0
384,https://www.cbf.com.br/futebol-brasileiro/comp...,CAU,Caucaia - CE,TOM,Tombense - MG,0 x 0,1ª Fase,Única,Copa do Brasil,"Qua, 22/02/2023 19:00\r\n - GRUPO 3...",João Ronaldo - Pacajus - CE\nComo foi o jogo,2023-02-22 19:00:00,2023.0,2.0,22.0,19.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,https://www.cbf.com.br/futebol-brasileiro/comp...,FLA,Flamengo - RJ,ATH,Athletico Paranaense - PR,Jogo ainda não ocorreu,Quartas de Final,Ida,Copa do Brasil,"Qua, 05/07/2023 21:30\r\n - GRU...",Maracanã - Rio de Janeiro - RJ\nDetalhes do jogo,2023-07-05 21:30:00,2023.0,7.0,5.0,21.0,30.0
476,https://www.cbf.com.br/futebol-brasileiro/comp...,GRE,Grêmio - RS,BAH,Bahia - BA,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qua, 12/07/2023 19:00\r\n -...",Arena do Grêmio - Porto Alegre - RS\nDetalhes ...,2023-07-12 19:00:00,2023.0,7.0,12.0,19.0,0.0
477,https://www.cbf.com.br/futebol-brasileiro/comp...,PAL,Palmeiras - SP,SAO,São Paulo - SP,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qui, 13/07/2023 20:00\r\n -...",Allianz Parque - Sao Paulo - SP\nDetalhes do jogo,2023-07-13 20:00:00,2023.0,7.0,13.0,20.0,0.0
478,https://www.cbf.com.br/futebol-brasileiro/comp...,COR,Corinthians - SP,AME,América Fc Saf - MG,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qua, 12/07/2023 21:30\r\n -...",Neo Química Arena - Sao Paulo - SP\nDetalhes d...,2023-07-12 21:30:00,2023.0,7.0,12.0,21.0,30.0


## Adicionar rodada do Brasileirão

In [811]:
base_jogos["# jogo"] = base_jogos["Data"].str.split("Jogo:").str[1] \
    .str.split(" \r\n                              \r\n                  ").str[0] \
    .str.split(" \r\n              - \r\n          ").str[0] \
    .str.split(" \r\n                      - \r\n              ").str[0] \
    .str.strip().astype("int")

base_jogos.loc[base_jogos["Torneio"] == "Brasileirão", "Rodada_temp"] = (((base_jogos.loc[base_jogos["Torneio"] == "Brasileirão", "# jogo"]/10).apply(np.ceil)).astype(str).str[:-2])
base_jogos.loc[base_jogos["Torneio"] == "Brasileirão", "Fase do torneio"] = "Rodada " + \
    base_jogos.loc[base_jogos["Torneio"] == "Brasileirão"].apply(lambda x: completa_digitos(x["Rodada_temp"], 2), axis = 1)

base_jogos = base_jogos.drop(["Rodada_temp"], axis = 1)

# base_jogos[(base_jogos["Torneio"] == "Brasileirão")]["Rodada_temp"]
# base_jogos[["Torneio", "Fase do torneio", "# jogo"]]
pd.DataFrame(base_jogos.groupby(["Torneio", "Fase do torneio"]).count().iloc[:, 0])

Link para o jogo
Torneio        Fase do torneio                   
Brasileirão    Rodada 01                       10
               Rodada 02                       10
               Rodada 03                       10
               Rodada 04                       10
               Rodada 05                       10
               Rodada 06                       10
               Rodada 07                       10
               Rodada 08                       10
               Rodada 09                       10
               Rodada 10                       10
               Rodada 11                       10
               Rodada 12                       10
               Rodada 13                       10
               Rodada 14                       10
               Rodada 15                       10
               Rodada 16                       10
               Rodada 17                       10
               Rodada 18                       10
               Rodada 19                       10
               Rodada 20                       10
               Rodada 21                       10
               Rodada 22                       10
               Rodada 23                       10
               Rodada 24                       10
               Rodada 25                       10
               Rodada 26                       10
               Rodada 27                       10
               Rodada 28                       10
               Rodada 29                       10
               Rodada 30                       10
               Rodada 31                       10
               Rodada 32                       10
               Rodada 33                       10
               Rodada 34                       10
               Rodada 35                       10
               Rodada 36                       10
               Rodada 37                       10
               Rodada 38                       10
Copa do Brasil 1ª Fase                         40
               2ª Fase                         20
               3ª Fase                         16
               Oitavas de Final                16
               Quartas de Final                 8

## Separar estado do time

In [812]:
# base_jogos.head()
base_jogos["Time mandante (UF)"] = base_jogos["Time mandante (nome)"].str[-2:].str.strip()
base_jogos["Time mandante (nome - UF)"] = base_jogos["Time mandante (nome)"].str.strip()
base_jogos["Time mandante (nome)"] = base_jogos["Time mandante (nome - UF)"].str[:-4].str.strip()

base_jogos[["Time mandante (nome - UF)", "Time mandante (nome)", "Time mandante (UF)"]]

,Time mandante (nome - UF),Time mandante (nome),Time mandante (UF)
0,América Fc Saf - MG,América Fc Saf,MG
1,Palmeiras - SP,Palmeiras,SP
2,Red Bull Bragantino - SP,Red Bull Bragantino,SP
3,Athletico Paranaense - PR,Athletico Paranaense,PR
4,Fortaleza - CE,Fortaleza,CE
...,...,...,...
475,Flamengo - RJ,Flamengo,RJ
476,Grêmio - RS,Grêmio,RS
477,Palmeiras - SP,Palmeiras,SP
478,Corinthians - SP,Corinthians,SP


In [813]:
base_jogos.groupby(["Time mandante (UF)", "Time mandante (nome)"]).count().index
# base_jogos.groupby(["Time mandante (nome)"]).count().index

# base_jogos[base_jogos["Time mandante (nome)"] == "Campinense"]

MultiIndex([('AC',                      'Humaitá'),
            ('AC',                'Sao Francisco'),
            ('AL',                          'Asa'),
            ('AL',                          'Crb'),
            ('AL',                          'Csa'),
            ('AM',         'Princesa do Solimões'),
            ('AP',                         'Trem'),
            ('BA',                     'Atlético'),
            ('BA',                        'Bahia'),
            ('BA',               'Bahia de Feira'),
            ('BA',                   'Jacuipense'),
            ('CE',                      'Caucaia'),
            ('CE',                    'Fortaleza'),
            ('CE',                       'Iguatu'),
            ('DF',                    'Ceilândia'),
            ('ES', 'Real Noroeste Capixaba F. C.'),
            ('ES',                'Vitoria F. C.'),
            ('GO',                     'Atlético'),
            ('GO',                        'Goiás'),
            

In [814]:
# base_jogos.head()
base_jogos["Time visitante (UF)"] = base_jogos["Time visitante (nome)"].str[-2:].str.strip()
base_jogos["Time visitante (nome - UF)"] = base_jogos["Time visitante (nome)"].str.strip()
base_jogos["Time visitante (nome)"] = base_jogos["Time visitante (nome - UF)"].str[:-4].str.strip()

base_jogos[["Time visitante (nome - UF)", "Time visitante (nome)", "Time visitante (UF)"]]

,Time visitante (nome - UF),Time visitante (nome),Time visitante (UF)
0,Fluminense - RJ,Fluminense,RJ
1,Cuiabá Saf - MT,Cuiabá Saf,MT
2,Bahia - BA,Bahia,BA
3,Goiás - GO,Goiás,GO
4,Internacional - RS,Internacional,RS
...,...,...,...
475,Athletico Paranaense - PR,Athletico Paranaense,PR
476,Bahia - BA,Bahia,BA
477,São Paulo - SP,São Paulo,SP
478,América Fc Saf - MG,América Fc Saf,MG


In [815]:
base_jogos.groupby(["Time visitante (UF)", "Time visitante (nome)"]).count().index

MultiIndex([('AL',                          'Crb'),
            ('AL',                          'Csa'),
            ('AM',                       'Manaus'),
            ('BA',                        'Bahia'),
            ('BA',                      'Vitória'),
            ('CE',                        'Ceará'),
            ('CE',                  'Ferroviário'),
            ('CE',                    'Fortaleza'),
            ('CE',                       'Iguatu'),
            ('DF',                  'Brasiliense'),
            ('GO',                     'Atlético'),
            ('GO',                        'Goiás'),
            ('GO',                    'Vila Nova'),
            ('MA',               'Sampaio Corrêa'),
            ('MG',               'América Fc Saf'),
            ('MG',             'Atlético Mineiro'),
            ('MG',                 'Cruzeiro Saf'),
            ('MG',                     'Tombense'),
            ('MS',                     'Operário'),
            

## Mais dados sobre local do jogo

In [816]:
base_jogos["Estádio"] = base_jogos["Local"].str.replace("\nComo foi o jogo", "").str.replace("\nDetalhes do jogo", "").str.split(" - ").str[0]
base_jogos["Cidade"] = base_jogos["Local"].str.replace("\nComo foi o jogo", "").str.replace("\nDetalhes do jogo", "").str.split(" - ").str[1]
base_jogos["UF"] = base_jogos["Local"].str.replace("\nComo foi o jogo", "").str.replace("\nDetalhes do jogo", "").str.split(" - ").str[2]

In [817]:
base_jogos.groupby(["UF", "Cidade", "Estádio"]).count().index

MultiIndex([('AC',            'Rio Branco',                'Florestão'),
            ('AL',             'Arapiraca',          'Coaracy Fonseca'),
            ('AL',                'Maceio',                 'Rei Pelé'),
            ('AM',            'Manacapuru',       'Gilberto Mestrinho'),
            ('BA',            'Alagoinhas',         'Antônio Carneiro'),
            ('BA',      'Feira de Santana',           'Arena Cajueiro'),
            ('BA',              'Salvador',         'Arena Fonte Nova'),
            ('BA',              'Salvador',                  'Pituaçu'),
            ('CE',             'Fortaleza',           'Arena Castelão'),
            ('CE',                'Iguatu',                  'Morenão'),
            ('CE',               'Pacajus',             'João Ronaldo'),
            ('DF',              'Brasilia', 'Boca do Jacaré / Serejão'),
            ('DF',              'Brasilia',           'Mané Garrincha'),
            ('ES',          'Aguia Branca',    'Jos

## Mais detalhes do resultado

In [818]:
base_jogos["Penaltis_temp"] =  base_jogos["Resultado"].str.find("(")
base_jogos.loc[base_jogos["Penaltis_temp"] < 0, "Houve penaltis?"] = (False)
base_jogos.loc[base_jogos["Penaltis_temp"] >= 0, "Houve penaltis?"] = (True)

display(base_jogos[["Resultado", "Penaltis_temp", "Houve penaltis?"]])

base_jogos = base_jogos.drop("Penaltis_temp", axis = 1)

,Resultado,Penaltis_temp,Houve penaltis?
0,0 x 3,-1,False
1,2 x 1,-1,False
2,2 x 1,-1,False
3,2 x 0,-1,False
4,1 x 1,-1,False
...,...,...,...
475,Jogo ainda não ocorreu,-1,False
476,Jogo ainda não ocorreu,-1,False
477,Jogo ainda não ocorreu,-1,False
478,Jogo ainda não ocorreu,-1,False


In [819]:
base_jogos.loc[base_jogos["Resultado"] != "Jogo ainda não ocorreu", "Gols mandante"] = \
    base_jogos.loc[base_jogos["Resultado"] != "Jogo ainda não ocorreu", "Resultado"].str.split(" x ").str[0].str[-1:].astype(int)
base_jogos.loc[base_jogos["Resultado"] == "Jogo ainda não ocorreu", "Gols mandante"] = np.nan

base_jogos.loc[base_jogos["Resultado"] != "Jogo ainda não ocorreu", "Gols visitante"] = \
    base_jogos.loc[base_jogos["Resultado"] != "Jogo ainda não ocorreu", "Resultado"].str.split(" x ").str[1].str[0].astype(int)
base_jogos.loc[base_jogos["Resultado"] == "Jogo ainda não ocorreu", "Gols visitante"] = np.nan


display(base_jogos[["Resultado", "Gols mandante", "Gols visitante"]])

,Resultado,Gols mandante,Gols visitante
0,0 x 3,0.0,3.0
1,2 x 1,2.0,1.0
2,2 x 1,2.0,1.0
3,2 x 0,2.0,0.0
4,1 x 1,1.0,1.0
...,...,...,...
475,Jogo ainda não ocorreu,NaN,NaN
476,Jogo ainda não ocorreu,NaN,NaN
477,Jogo ainda não ocorreu,NaN,NaN
478,Jogo ainda não ocorreu,NaN,NaN


In [ ]:
base_jogos.loc[base_jogos["Resultado"] != "Jogo ainda não ocorreu", "Resultado (sem penaltis)"] = \
    base_jogos.loc[base_jogos["Resultado"] != "Jogo ainda não ocorreu", "Gols mandante"].astype(str).str[0] \
    + " x " + \
    base_jogos.loc[base_jogos["Resultado"] != "Jogo ainda não ocorreu", "Gols visitante"].astype(str).str[0]

base_jogos.loc[base_jogos["Resultado"] == "Jogo ainda não ocorreu", "Resultado (sem penaltis)"] = "Jogo ainda não ocorreu"

display(base_jogos[["Resultado", "Resultado (sem penaltis)"]].head())
display(base_jogos[base_jogos["Houve penaltis?"]][["Resultado", "Resultado (sem penaltis)"]].head())
display(base_jogos[base_jogos["Resultado"] == "Jogo ainda não ocorreu"][["Resultado", "Resultado (sem penaltis)"]].head())

,Resultado,Resultado (sem penaltis)
0,0 x 3,0 x 3
1,2 x 1,2 x 1
2,2 x 1,2 x 1
3,2 x 0,2 x 0
4,1 x 1,1 x 1


,Resultado,Resultado (sem penaltis)
428,(6)1 x 1(5),1 x 1
431,(4)1 x 1(2),1 x 1
432,(7)0 x 0(6),0 x 0
433,(4)1 x 1(5),1 x 1
439,(5)0 x 0(6),0 x 0


,Resultado,Resultado (sem penaltis)
110,Jogo ainda não ocorreu,Jogo ainda não ocorreu
111,Jogo ainda não ocorreu,Jogo ainda não ocorreu
112,Jogo ainda não ocorreu,Jogo ainda não ocorreu
113,Jogo ainda não ocorreu,Jogo ainda não ocorreu
114,Jogo ainda não ocorreu,Jogo ainda não ocorreu


In [820]:
base_jogos.loc[base_jogos["Houve penaltis?"], "Penaltis mandante"] = \
    base_jogos[base_jogos["Houve penaltis?"]]["Resultado"].str.split(")").str[0].str.replace("(", "").astype(int)
base_jogos.loc[base_jogos["Houve penaltis?"] == False, "Penaltis mandante"] = np.nan

base_jogos.loc[base_jogos["Houve penaltis?"], "Penaltis visitante"] = \
    base_jogos[base_jogos["Houve penaltis?"]]["Resultado"].str.split("(").str[2].str.replace(")", "").astype(int)
base_jogos.loc[base_jogos["Houve penaltis?"] == False, "Penaltis visitante"] = np.nan

display(base_jogos[["Resultado", "Penaltis mandante", "Penaltis visitante"]])

,Resultado,Penaltis mandante,Penaltis visitante
0,0 x 3,NaN,NaN
1,2 x 1,NaN,NaN
2,2 x 1,NaN,NaN
3,2 x 0,NaN,NaN
4,1 x 1,NaN,NaN
...,...,...,...
475,Jogo ainda não ocorreu,NaN,NaN
476,Jogo ainda não ocorreu,NaN,NaN
477,Jogo ainda não ocorreu,NaN,NaN
478,Jogo ainda não ocorreu,NaN,NaN


In [839]:
base_jogos.loc[base_jogos["Houve penaltis?"], "Resultado dos penaltis"] = \
    base_jogos.loc[base_jogos["Houve penaltis?"], "Penaltis mandante"].astype(str).str[0] \
    + " x " + \
    base_jogos.loc[base_jogos["Houve penaltis?"], "Penaltis visitante"].astype(str).str[0]

base_jogos.loc[base_jogos["Houve penaltis?"] == False, "Resultado dos penaltis"] == np.nan

display(base_jogos[base_jogos["Houve penaltis?"]][["Resultado", "Resultado dos penaltis"]].head())
display(base_jogos[base_jogos["Houve penaltis?"] == False][["Resultado", "Resultado dos penaltis"]].head())

,Resultado,Resultado dos penaltis
428,(6)1 x 1(5),6 x 5
431,(4)1 x 1(2),4 x 2
432,(7)0 x 0(6),7 x 6
433,(4)1 x 1(5),4 x 5
439,(5)0 x 0(6),5 x 6


,Resultado,Resultado dos penaltis
0,0 x 3,NaN
1,2 x 1,NaN
2,2 x 1,NaN
3,2 x 0,NaN
4,1 x 1,NaN


# Análise exploratória

# Preparação de dados

## One hot encoding

## Balanceamento

## Separação treino e teste

## Normalização

# Classificação/Regressão

# TO-DO